tree:
.
├── data/
│   └── Research Data Project/
│       └── exit_velo_project_data.csv
├── src/
│   ├── data/
│   │   ├── load_data.py
│   │   └── ColumnSchema.py
│   ├── features/
│   │   ├── data_prep.py
│   │   ├── eda.py
│   │   ├── feature_engineering.py
│   │   ├── feature_selection.py
│   │   ├── preprocess.py
│   │   └── __init__.py
│   ├── models/
│   │   ├── bayesian_alternatives.py
│   │   ├── gbm.py
│   │   ├── hierarchical.py
│   │   ├── linear.py
│   │   ├── mixed.py
│   │   └── hierarchical_predict.py
│   ├── utils/
│   │   ├── bayesian_explainability.py
│   │   ├── bayesian_metrics.py
│   │   ├── gbm_utils.py
│   │   ├── hierarchical_utils.py
│   │   ├── jax_gpu_utils.py
│   │   ├── jax_memory_monitor.py
│   │   ├── posterior.py
│   │   ├── validation.py
│   │   └── __init__.py
│   └── train.py
└── requirements.txt





src/data

    load_data.py
    • load_data: Reads the raw CSV of exit-velo observations.
    • clean_raw: Drops rows missing the target (exit_velo) or key columns, reports remaining nulls.
    • load_and_clean_data: Convenience wrapper that runs both in sequence.
    Purpose: Centralize raw-data ingestion and initial cleanup so downstream code always sees a non-mutated, null-filtered DataFrame.

    ColumnSchema.py
    • Defines _ColumnSchema, with lists of ID, ordinal, nominal, numerical, and target columns.
    • Exposes methods like .numerical(), .categorical(), .model_features() for type-safe feature lists.
    Purpose: Avoid hard-coding column names everywhere—everything downstream (preprocessing, modeling) refers to this single source of truth.

src/features

    feature_engineering.py
    • Pure functions that add derived columns:
    – binned features (age_bin, la_bin, spray_bin)
    – rolling, lagged statistics (player_ev_mean50, etc.)
    – categorical flags (same_hand, hitter_type)
    Purpose: Enrich raw data with predictive covariates while avoiding leakage.

    eda.py
    • Quick checks and diagnostics: null summaries, correlation tables, ANOVA for league-level effects, LOWESS age trends, Durbin–Watson tests, distribution plots, etc.
    Purpose: Exploratory-data-analysis helpers to understand data shape, spot red flags (small samples, outliers), and guide model design (e.g. hierarchical effects).

    data_prep.py
    • Low-level utilities for domain-specific filtering & clipping:
    – compute_clip_bounds
    – clip_extreme_ev (trim 1st/99th-pct EVs)
    – filter_bunts_and_popups, filter_low_event_batters, filter_physical_implausibles
    Purpose: Apply baseball-specific rules to remove implausible or uninformative observations before modeling.

    preprocess.py
    • Builds full scikit-learn pipelines:
    – filter_and_clip for domain cleaning
    – fit_preprocessor/transform_preprocessor to impute, encode, scale, and preserve clipping bounds
    – inverse_transform_preprocessor to map transformed arrays back to original features
    – prepare_for_mixed_and_hierarchical adds category codes for mixed-effects and Bayesian models
    Purpose: Turn your feature-engineered DataFrame into numeric arrays ready for any downstream estimator, with consistent handling of missingness and train/test separation.

    feature_selection.py
    • Baseline RandomForest (train_baseline_model)
    • Permutation importance (compute_permutation_importance) and SHAP importance (compute_shap_importance)
    • Helpers to threshold and intersect feature lists, plus I/O for saving/loading your final feature list
    Purpose: Automate ranking and selection of the most predictive features before you commit to a final modeling dataset.

src/models

    linear.py
    • OLS/Ridge regression wrappers with simple fit_ridge interface.
    Purpose: Quick, interpretable baselines for exit-velo prediction.

    gbm.py
    • XGBoost regressor pipeline with GPU detection, Optuna-based tuning (tune_gbm), early stopping and save/load via gbm_utils.
    Purpose: High-performing gradient-boosting baseline that scales to larger datasets.

    mixed.py
    • Frequentist mixed-effects model (statsmodels.MixedLM) capturing random intercepts by batter.
    Purpose: Model batter-level variability explicitly, bridging simple regressions and full Bayesian hierarchies.

    hierarchical.py
    • Full Bayesian hierarchical model in PyMC (optionally accelerated via JAX).
    • Priors on global mean, level effects, batter/season/pitcher random effects, plus memory-monitoring hooks.
    Purpose: Capture multi-level structure (league, batter, season, pitcher) and quantify uncertainty via full posterior inference.

    bayesian_alternatives.py
    • Plug-and-play interfaces for other Bayesian engines: CmdStanPy, PyJAGS, NumPyro, TensorFlow-Probability, PyMC-ADVI.
    Purpose: Compare different probabilistic back-ends under the same InferenceData API.

    hierarchical_predict.py
    • Post-hoc prediction utilities that merge saved global effects JSON and posterior summaries with a new roster to produce point forecasts and intervals for 2024.
    Purpose: Turn your trained hierarchical model into actionable predictions on fresh data.

    model_shap_reports.py
    • Wrappers around ExplainerDashboard (Dash), Shapash, and Shapiq for interactive and static explainability reports.
    Purpose: Provide end-to-end explainability for your chosen model.

src/utils

    gbm_utils.py: save/load combined GBM + preprocessor pipelines via joblib.

    bayesian_explainability.py: Arviz forest plots, posterior-predictive checks, plus kernel SHAP on posterior means.

    bayesian_metrics.py: Compute classical (RMSE, MAE, R²) and Bayesian model‐comparison metrics (LOO, WAIC).

    hierarchical_utils.py: Save/load ArviZ InferenceData to NetCDF and preprocessor joblib.

    posterior.py: Extract per-batter random‐effect summaries into a DataFrame, and write global effects out to JSON.

    validation.py: K-fold CV for both sklearn estimators and PyMC models, plus prediction‐interval helpers (analytic and bootstrap).

    jax_gpu_utils.py: GPU diagnostics for JAX backends (checks nvidia-smi, device list, LD_LIBRARY_PATH).

    (plus memory-monitoring and JAX memory-fix modules)

Purpose: Miscellaneous building blocks for model persistence, diagnostics, metrics, and workflow orchestration.
Top-Level Scripts

    train.py
    Orchestrates a 70/30 split to fit and compare Ridge, GBM, mixed-LM, and quick Bayesian HMC, printing RMSEs side by side.
    Purpose: Unified entry point to benchmark all four modeling paradigms on exit-velo data.

Overall Pipeline Flow

    Load & clean raw CSV → 2. Feature-engineer (new covariates) → 3. Preprocess (impute, encode, scale, clip) → 4. Select top features → 5. Fit model(s) → 6. Evaluate & compare metrics → 7. Explain results → 8. Predict on 2024 roster with saved summaries.

Each module slots neatly into one of those stages, making your codebase both modular and extensible for future seasons or new model families.

In [7]:
%%writefile src/data/load_data.py
import pandas as pd
import numpy as np

def load_data(path='data/Research Data Project/Research Data Project/exit_velo_project_data.csv'):
    """
    Load raw exit velocity data from the data directory.
    
    Returns:
        pd.DataFrame: Raw exit velocity data
    """

    df = pd.read_csv(path)
    return df 



# ──  utility ─────────────────────────────────────────────────────────
def clean_raw(df: pd.DataFrame, 
              target: str = "exit_velo"
              ,debug: bool = False) -> pd.DataFrame:
    """
    Central place to:
      1. Drop rows with NaN in *target*.
      2. Print a concise null‑summary afterwards (one line per column).
    
    Returns a *fresh copy* (never mutates in‑place).
    """
    if debug:
        print(f"Dropping rows with NaN in {target}")
        print(df.isna().sum())
        
    drop_columns = [target, "hit_type", "launch_angle"]
    out = df.dropna(subset=drop_columns).copy()   

    if debug:
        print(f"after rows dropped with NaN in {target}")
        print(out.isna().sum())
    # quick dashboard
    nulls = out.isna().sum()
    non_zero = nulls[nulls > 0]
    if non_zero.empty:
        print(f"✅  After target‑filter, no other nulls (n={len(out):,}).")
    else:
        print("⚠️  Nulls after target‑filter:")
        for col, cnt in non_zero.items():
            pct = cnt / len(out)
            print(f"  • {col:<15} {cnt:>7,} ({pct:5.2%})")

    return out




def load_and_clean_data(path='data/Research Data Project/Research Data Project/exit_velo_project_data.csv'
                        ,debug: bool = False):
    df = load_data(path)
    df = clean_raw(df, debug = True) 
    return df


if __name__ == "__main__":
    path = 'data/Research Data Project/Research Data Project/exit_velo_project_data.csv'

    df = load_and_clean_data(path, debug = True)
    print(df.head())
    print(df.columns)
    

Overwriting src/data/load_data.py


# Feature Engineering

In [8]:
%%writefile src/features/feature_engineering.py
"""Feature engineering utilities for the exit‑velo project.

All helpers are pure functions that take a pandas DataFrame and return a *copy*
with additional engineered columns so we avoid side effects.

"""
from __future__ import annotations

import pandas as pd
import numpy as np

###############################################################################
# Helper functions
###############################################################################
def _classify_hitters_by_season(
    df: pd.DataFrame,
    batter_col: str = "batter_id",
    season_col: str = "season",
    outcome_col: str = "outcome",
    power_pct: float = 0.8,
) -> pd.Series:
    """
    Season-by-season classification: top `power_pct` hitters by triple+HR rate
    are 'POWER', all others 'CONTACT'. Batters with no hits default to CONTACT.
    """
    # 1) restrict to actual base hits
    hits = df[df[outcome_col].isin(["SINGLE","DOUBLE","TRIPLE","HOME RUN"])].copy()

    # 2) count per (season, batter)
    counts = (
        hits
        .assign(
            contact = lambda d: d[outcome_col].isin(["SINGLE","DOUBLE"]).astype(int),
            power   = lambda d: d[outcome_col].isin(["TRIPLE","HOME RUN"]).astype(int),
        )
        .groupby([season_col, batter_col])
        .agg(
            total_hits   = (outcome_col, "size"),
            contact_hits = ("contact",    "sum"),
            power_hits   = ("power",      "sum"),
        )
    )
    counts["power_rate"]   = counts["power_hits"]   / counts["total_hits"]
    counts["contact_rate"] = counts["contact_hits"] / counts["total_hits"]

    # 3) find the season‐level 80th percentile of power_rate
    pct80 = counts.groupby(level=0)["power_rate"].quantile(power_pct)
    # map it back onto each row
    counts["season_power_80"] = counts.index.get_level_values(season_col).map(pct80)

    # 4) label
    def _label(row):
        return "POWER" if row.power_rate >= row.season_power_80 else "CONTACT"

    labels = counts.apply(_label, axis=1)
    labels.name = "hitter_type"
    return labels




def _rolling_stat_lagged(
    df: pd.DataFrame,
    group_cols: list[str],
    target: str,
    stat: str = "mean",
    window: int = 50,
) -> pd.Series:
    """
    Group-wise rolling statistic using only *previous* rows.
    For each group defined by group_cols, shift the target by 1 row 
    then compute a rolling(window) agg(stat) with min_periods=10.
    """
    # 1) Within each group, shift the target by one so we only use past data
    def shifted_rolling(x: pd.Series) -> pd.Series:
        return x.shift(1).rolling(window=window, min_periods=10).agg(stat)

    rolled = (
        df
        .groupby(group_cols)[target]     # group by batter_id or pitcher_id
        .apply(shifted_rolling)          # shift & roll inside each group
        .reset_index(level=group_cols, drop=True)  # get back a plain Series
    )
    return rolled


###############################################################################
# Public API
###############################################################################

def feature_engineer(df: pd.DataFrame, copy: bool = True) -> pd.DataFrame:
    """Return a DataFrame enriched with engineered features (no leakage)."""

    if copy:
        df = df.copy()

    # 1) Uppercase strings
    str_cols = df.select_dtypes(include=['object', 'string']).columns
    df[str_cols] = df[str_cols].apply(lambda col: col.str.upper())

    # 2) Age
    df["age_sq"]   = df["age"] ** 2
    df["age_bin"]  = pd.qcut(df["age"], q=4, duplicates='drop')

    # 3) Height
    avg_height    = df["batter_height"].mean()
    df["height_diff"] = df["batter_height"] - avg_height

    # 4) Launch & spray bins
    df["la_bin"]    = pd.qcut(df["launch_angle"], q=5, duplicates='drop')
    df["spray_bin"] = pd.qcut(df["spray_angle"], q=3, duplicates='drop')

    # 5) Handedness & matchups
    df["same_hand"]        = (df["batter_hand"] == df["pitcher_hand"])
    df["hand_match"]       = df["batter_hand"] + "_VS_" + df["pitcher_hand"]
    df["pitch_hand_match"] = df["pitch_group"] + "_" + df["hand_match"]

    # 6) Batter lagged rolling EV stats
    df["player_ev_mean50"] = _rolling_stat_lagged(df, ["batter_id"], "exit_velo", "mean", 50)
    df["player_ev_std50"]  = _rolling_stat_lagged(df, ["batter_id"], "exit_velo",  "std", 50)
    gm = df["exit_velo"].mean(); gs = df["exit_velo"].std()
    df["player_ev_mean50"].fillna(gm)
    df["player_ev_std50"].fillna(gs)

    # 7) Pitcher lagged mean
    df["pitcher_ev_mean50"] = _rolling_stat_lagged(df, ["pitcher_id"], "exit_velo", "mean", 50)
    df["pitcher_ev_mean50"].fillna(gm)

    # 8) Center covariates
    df["age_centered"]    = df["age"]    - df["age"].median()
    df["season_centered"] = df["season"] - df["season"].median()
    df["level_idx"]       = df["level_abbr"].map({"AA":0, "AAA":1, "MLB":2})

    # --- 9) New season‐by‐season batter classification ---
    labels = _classify_hitters_by_season(df)
    # bring labels into df by season & batter_id
    labels_df = labels.reset_index()  # columns: [season, batter_id, hitter_type]
    df = df.merge(labels_df, on=["season","batter_id"], how="left")

    # 10) fill anyone still NaN → they had no base hits
    df["hitter_type"] = df["hitter_type"].fillna("CONTACT")
    return df

###############################################################################
# CLI entry‑point (quick smoke test)
###############################################################################

if __name__ == "__main__":
    from pathlib import Path
    from src.data.load_data import load_and_clean_data

    raw_path = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    df = load_and_clean_data(raw_path, debug = True)
    print(df.head())
    print(df.columns)

    df_fe = feature_engineer(df)

    print("Raw →", df.shape, "//  Feature‑engineered →", df_fe.shape)
    print(df_fe.head())
    print(df_fe.columns)
    
            
    # --- DEBUG: batters with no classification ---
    missing_mask = df_fe["hitter_type"].isna()
    missing_df   = df_fe[missing_mask].copy()
    unique_b     = missing_df["batter_id"].nunique()
    print(f"[DEBUG] {unique_b} unique batters have no label (NaN in hitter_type).")

    # --- Fix sort_values key to map each array's length ---
    outcome_by_batter = (
        missing_df
        .groupby("batter_id")["outcome"]
        .unique()
        .sort_values(
            key=lambda s: s.map(len),      # for each entry, use len(array)
            ascending=False
        )
    )
    print("[DEBUG] Sample of missing batters → their unique outcomes:")
    print(outcome_by_batter.head(10).to_dict())

    print("[DEBUG] Outcome value counts among missing batters:")
    print(missing_df["outcome"].value_counts().to_dict())


Overwriting src/features/feature_engineering.py


# ColumnSchema: separates raw and engineered columns

In [9]:
%%writefile src/data/ColumnSchema.py
""" 
Column schema helper for exit‑velo project.

Centralises every raw and engineered column name in one place and exposes
 type‑safe accessors so downstream code never hard‑codes strings.

Usage
-----
>>> from src.features.columns import cols
>>> num_cols = cols.numerical()
>>> ord_cols = cols.ordinal()
>>> all_for_model = cols.model_features()
"""

from functools import lru_cache
from typing import List, Dict
import json

class _ColumnSchema:
    """Container for canonical column lists.

    Keeping everything behind methods avoids accidental mutation and lets
    IDEs offer autocompletion (because the return type is always `List[str]`).
    """

    _ID_COLS: List[str] = [
        "season", "batter_id", "pitcher_id",
    ]

    _ORDINAL_CAT_COLS: List[str] = [
        "level_abbr",   # AA < AAA < MLB
        "age_bin",      # 4 quantile bins of age
        "la_bin",       # 5 quantile bins of launch angle
        "spray_bin",    # 3 quantile bins of spray angle
        # "outcome",      # categorical outcome (out, single, double, triple, home run, sacrifice fly, sacrifice bunt, fielders choice)
    ]


    _NOMINAL_CAT_COLS = [
        "hit_type",
        "pitch_group",
        "batter_hand",
        "pitcher_hand",
        "hand_match",
        "pitch_hand_match",
        "same_hand",
        "hitter_type", 
    ]

    _NUMERICAL_COLS = [
        "launch_angle",
        "spray_angle",
        "hangtime",
        "height_diff",
        "age_sq",
        "age_centered",
        "season_centered",
        "level_idx",
        "player_ev_mean50",
        "player_ev_std50",
        "pitcher_ev_mean50",
        # "power_rate",
        # "season_power_80", 
    ]


    _TARGET_COL: str = "exit_velo"

    # ────────────────────────────────────────────────────────────────────
    # Public helpers
    # ────────────────────────────────────────────────────────────────────
    def id(self) -> List[str]:
        return self._ID_COLS.copy()

    def ordinal(self) -> List[str]:
        return self._ORDINAL_CAT_COLS.copy()

    def nominal(self) -> List[str]:
        return self._NOMINAL_CAT_COLS.copy()

    def categorical(self) -> List[str]:
        """All cat cols (ordinal + nominal)."""
        return self._ORDINAL_CAT_COLS + self._NOMINAL_CAT_COLS

    def numerical(self) -> List[str]:
        return self._NUMERICAL_COLS.copy()


    def target(self) -> str:
        return self._TARGET_COL

    # ------------------------------------------------------------------
    @lru_cache(maxsize=1)
    def model_features(self) -> List[str]:
        """Columns fed into the ML pipeline *after* preprocess.

        Excludes the target but includes derived cols.
        """
        phys_minus_target = [
            c for c in self._NUMERICAL_COLS if c != self._TARGET_COL
        ]

        return (
            phys_minus_target
            + self._NOMINAL_CAT_COLS
            + self._ORDINAL_CAT_COLS  # some algos want raw string order
        )

    def all_raw(self) -> List[str]:
        """Returns every column expected in raw input CSV (incl. engineered)."""
        return (
            self._ID_COLS
            + self._ORDINAL_CAT_COLS
            + self._NOMINAL_CAT_COLS
            + self._NUMERICAL_COLS
        )

    def as_dict(self) -> Dict[str, List[str]]:
        """Dictionary form – handy for YAML/JSON dumps."""
        return {
            "id": self.id(),
            "ordinal": self.ordinal(),
            "nominal": self.nominal(),
            "numerical": self.numerical(),
            "target": [self.target()],
        }


if __name__ == "__main__":
    cols = _ColumnSchema()

    __all__ = ["cols"]
    print("ID columns:         ", cols.id())
    print("Ordinal columns:    ", cols.ordinal())
    print("Nominal columns:    ", cols.nominal())
    print("All categorical:    ", cols.categorical())
    print("Numerical columns:  ", cols.numerical())
    print("Target column:      ", cols.target())
    print("Model features:     ", cols.model_features())
    print("All raw columns:    ", cols.all_raw())
    print("\nAs dict (JSON):")
    print(json.dumps(cols.as_dict(), indent=2))



Overwriting src/data/ColumnSchema.py


In [10]:
%%writefile src/features/eda.py

import pandas as pd  # type: ignore
import matplotlib.pyplot as plt  # type: ignore
import numpy as np  # type: ignore
from src.data.ColumnSchema import _ColumnSchema
from pandas.api.types import is_categorical_dtype
from scipy.stats import f_oneway, ttest_ind

# Optional imports with fallbacks for advanced statistics
try:
    import scipy.stats as stats  # type: ignore
    from statsmodels.nonparametric.smoothers_lowess import (
        lowess  # type: ignore
    )
    from statsmodels.stats.stattools import (
        durbin_watson  # type: ignore
    )
    _HAS_STATS_LIBS = True
except ImportError:
    _HAS_STATS_LIBS = False
    print("Warning: scipy or statsmodels not available. "
          "Some diagnostics will be limited.")



def get_column_groups() -> dict:
    """
    Return a mapping of column-type → list of columns,
    based on the canonical schema in src.features.feature_selection.cols.
    """
    return cols.as_dict()

def check_nulls(df: pd.DataFrame):
    # Identify columns with null values
    null_columns = df.columns[df.isnull().any()].tolist()
    
    # Output the columns with null values
    if null_columns:
        print("Columns with null values:", null_columns)
    else:
        print("No columns with null values.")


def quick_pulse_check(
    df: pd.DataFrame,
    velo_col: str = "exit_velo",
    group_col: str = "batter_id",
    level_col: str = "level_abbr"
) -> pd.DataFrame:
    """
    Print a quick summary table:
      - total rows
      - unique batters
      - overall median exit_velo
      - median exit_velo by level
      - distribution of events per batter (median, 25th pct)
      - distribution of seasons per batter
      - pearson correlations of velo with launch_angle & hangtime
    Returns a pd.DataFrame with those metrics.
    """
    df = df.copy()
    total_rows = len(df)
    n_batters = df[group_col].nunique()
    overall_med = df[velo_col].median()

    # median by level
    med_by_level = df.groupby(level_col)[velo_col].median()

    # events per batter
    ev_per = df[group_col].value_counts()
    ev_stats = ev_per.quantile([0.25, 0.5]).to_dict()

    # seasons per batter
    seasons_per = df.groupby(group_col)["season"].nunique()
    seasons_stats = seasons_per.value_counts().sort_index().to_dict()

    # basic correlations
    corr = df[[velo_col, "launch_angle", "hangtime"]].corr()[velo_col].drop(velo_col)

    # Build a summary table
    metrics = [
        "Total rows",
        "Unique batters",
        "Overall median EV",
    ]
    values = [
        total_rows,
        n_batters,
        overall_med,
    ]
    
    # Add level-specific metrics
    for lvl in med_by_level.index:
        metrics.append(f"Median EV @ {lvl}")
        values.append(med_by_level[lvl])
    
    # Add batter event metrics
    metrics.extend([
        "Events per batter (25th pct)",
        "Events per batter (median)",
    ])
    values.extend([
        ev_stats.get(0.25, "N/A"),
        ev_stats.get(0.5, "N/A"),
    ])
    
    # Add season distribution
    for season_count, count in seasons_stats.items():
        metrics.append(f"Batters with {season_count} season(s)")
        values.append(count)
    
    # Add correlations
    metrics.extend([
        "ρ(exit_velo, launch_angle)",
        "ρ(exit_velo, hangtime)",
    ])
    values.extend([
        corr.get("launch_angle", "N/A"),
        corr.get("hangtime", "N/A"),
    ])

    table = pd.DataFrame({
        "Metric": metrics,
        "Value": values
    })
    
    print(table.to_string(index=False))
    return table


def red_flag_small_samples(df: pd.DataFrame,
                           group_col: str = "batter_id",
                           threshold: int = 15) -> pd.Series:
    """
    Identify batters with fewer than `threshold` events.
    Returns a Series of counts indexed by batter_id.
    """
    counts = df[group_col].value_counts()
    small = counts[counts < threshold]
    print(f"> Batters with fewer than {threshold} events: {len(small)}")
    if len(small) > 0:
        print(f"  First few: {', '.join(map(str, small.index[:5]))}")
    return small


def red_flag_level_effect(df: pd.DataFrame,
                          level_col: str = "level_abbr",
                          velo_col: str = "exit_velo") -> tuple:
    """
    One-way ANOVA of exit_velo across levels.
    Returns (F-statistic, p-value) or (None, None) if scipy is not available.
    """
    if not _HAS_STATS_LIBS:
        print("> ANOVA on exit_velo by level: scipy not available")
        print("> Basic level summary instead:")
        summary = df.groupby(level_col)[velo_col].agg(['mean', 'std', 'count'])
        print(summary)
        return None, None
    
    groups = [
        df[df[level_col] == lvl][velo_col].dropna()
        for lvl in df[level_col].unique()
    ]
    F, p = stats.f_oneway(*groups)
    print(f"> ANOVA on {velo_col} by {level_col}: F={F:.3f}, p={p:.3e}")
    return F, p


# ------------------------------------------------------------------
#  REPLACE old red_flag_level_effect  → clearer name & doc
# ------------------------------------------------------------------
def league_level_effect(
    df: pd.DataFrame,
    level_col: str = "level_abbr",
    velo_col: str = "exit_velo",
) -> tuple[float | None, float | None]:
    """
    🔹 Why it matters – confirms MLB vs Triple‑A (etc.) differences to
      justify hierarchical level effects in the model.

    One‑way ANOVA of `exit_velo` across `level_col`.
    Returns (F, p) or (None, None) if SciPy unavailable.
    """
    if not _HAS_STATS_LIBS:
        print("> SciPy unavailable – falling back to group summary")
        print(df.groupby(level_col)[velo_col].describe())
        return None, None

    groups = [df[df[level_col] == lv][velo_col].dropna()
              for lv in df[level_col].unique()]
    f_val, p_val = stats.f_oneway(*groups)
    print(f"> Level effect ANOVA: F={f_val:.3f}, p={p_val:.3e}")
    return f_val, p_val



def diag_age_effect(df: pd.DataFrame,
                    age_col: str = "age_centered",
                    velo_col: str = "exit_velo") -> np.ndarray | None:
    """
    LOWESS smoothing of exit_velo vs. age_centered.
    Returns the smoothed array or None if statsmodels is not available.
    """
    if not _HAS_STATS_LIBS:
        print("> Age effect analysis: statsmodels not available")
        print("> Basic correlation instead:")
        corr = df[[age_col, velo_col]].corr().iloc[0, 1]
        print(f"Correlation between {age_col} and {velo_col}: {corr:.3f}")
        return None
    
    # Run LOWESS smoothing
    smooth_result = lowess(df[velo_col], df[age_col])
    
    # Plot the result
    plt.figure(figsize=(6, 3))
    plt.scatter(df[age_col], df[velo_col], alpha=0.1, s=1, color='gray')
    plt.plot(
        smooth_result[:, 0], 
        smooth_result[:, 1], 
        'r-', 
        linewidth=2, 
        label="LOWESS fit"
    )
    plt.xlabel(age_col)
    plt.ylabel(velo_col)
    plt.title("Age effect (LOWESS)")
    plt.legend()
    plt.tight_layout()
    
    return smooth_result


def diag_time_series_dw(
    df: pd.DataFrame,
    time_col: str = "season",
    group_col: str = "batter_id",
    velo_col: str = "exit_velo"
) -> pd.Series | None:
    """
    Compute Durbin–Watson on each batter's time series of mean exit_velo.
    Returns a Series of DW statistics or None if statsmodels is not available.
    """
    if not _HAS_STATS_LIBS:
        print("> Time series analysis: statsmodels not available")
        return None
    
    # Create pivot table of seasons (columns) by batters (rows)
    pivot = (
        df
        .groupby([group_col, time_col])[velo_col]
        .mean()
        .unstack(fill_value=np.nan)
    )
    
    # Only process batters with at least 3 seasons
    valid_batters = pivot.dropna(thresh=3).index
    if len(valid_batters) == 0:
        print("> No batters with sufficient seasons for Durbin-Watson test")
        return None
    
    # Calculate DW statistic for each valid batter
    dw_stats = {}
    for batter in valid_batters:
        series = pivot.loc[batter].dropna()
        if len(series) >= 3:  # Recheck after dropna
            dw = durbin_watson(series)
            dw_stats[batter] = dw
    
    dw_series = pd.Series(dw_stats)
    print(
        f"> Mean Durbin–Watson across {len(dw_series)} batters: "
        f"{dw_series.mean():.3f}"
    )
    print("> DW < 1.5 suggests positive autocorrelation")
    print("> DW > 2.5 suggests negative autocorrelation")
    print("> DW ≈ 2.0 suggests no autocorrelation")
    
    return dw_series


# ------------------------------------------------------------------
#  REPLACE old diag_time_series_dw WITH optional helper
# ------------------------------------------------------------------
def _optional_dw_check(
    df: pd.DataFrame,
    time_col: str = "season",
    group_col: str = "batter_id",
    velo_col: str = "exit_velo",
) -> pd.Series | None:
    """
    (OPTIONAL) Durbin–Watson residual autocorrelation **per batter**.
    Mostly irrelevant for cross‑sectional EV analysis but retained
    behind a private name for power users.
    """
    if not _HAS_STATS_LIBS:
        return None
    pivot = (
        df.groupby([group_col, time_col])[velo_col]
          .mean().unstack()
    )
    stats_out = {}
    for idx, row in pivot.dropna(thresh=3).iterrows():
        if row.count() >= 3:
            stats_out[idx] = durbin_watson(row.dropna())
    if not stats_out:
        print("> DW check: no eligible batters")
        return None
    s = pd.Series(stats_out)
    print(f"DW mean={s.mean():.2f} (1.5<→pos autocorr, >2.5→neg)")
    return s




def check_red_flags(df: pd.DataFrame, 
                    sample_threshold: int = 15) -> dict:
    """
    Run all red flag checks and return the results in a dictionary.
    """
    results = {}
    
    # Check for small sample sizes
    small_samples = red_flag_small_samples(df, threshold=sample_threshold)
    results['small_samples'] = small_samples
    
    # Check for level effects
    f_stat, p_val = red_flag_level_effect(df)
    results['level_effect'] = {
        'f_statistic': f_stat,
        'p_value': p_val
    }
    
    return results


def plot_distributions(df: pd.DataFrame,
                       velo_col: str = "exit_velo",
                       by: str = "level_abbr"):
    """
    Histogram of `velo_col` faceted by `by`.
    Returns the Matplotlib figure so callers can save or show it.
    """
    groups = df[by].unique()
    fig, axes = plt.subplots(len(groups), 1,
                             figsize=(6, 2.8 * len(groups)),
                             sharex=True)
    for ax, grp in zip(axes, groups):
        ax.hist(df[df[by] == grp][velo_col], bins=30, alpha=0.75)
        ax.set_title(f"{by} = {grp} (n={len(df[df[by] == grp])})")
        ax.set_xlabel(velo_col)
    fig.tight_layout()
    return fig


def plot_correlations(df: pd.DataFrame, cols: list[str]):
    """
    Heat-map of Pearson correlations for `cols`.
    """
    corr = df[cols].corr()
    fig, ax = plt.subplots(figsize=(0.6 * len(cols) + 2,
                                    0.6 * len(cols) + 2))
    im = ax.imshow(corr, vmin=-1, vmax=1, cmap="coolwarm")
    fig.colorbar(im, ax=ax, shrink=0.8)
    ax.set_xticks(range(len(cols)), cols, rotation=90)
    ax.set_yticks(range(len(cols)), cols)
    fig.tight_layout()
    return fig


def plot_time_trends(df: pd.DataFrame,
                     time_col: str = "season",
                     group_col: str = "batter_id",
                     velo_col: str = "exit_velo",
                     sample: int = 50):
    """
    Plot mean exit-velo over time for a random sample of batters.
    """
    batters = df[group_col].unique()
    chosen = np.random.choice(batters,
                              min(sample, len(batters)),
                              replace=False)
    fig, ax = plt.subplots(figsize=(8, 4))
    for b in chosen:
        series = (
            df[df[group_col] == b]
            .groupby(time_col)[velo_col]
            .mean()
        )
        ax.plot(series.index, series.values, alpha=0.3)
    ax.set_xlabel(time_col)
    ax.set_ylabel(velo_col)
    ax.set_title("Sample batter exit-velo over time")
    fig.tight_layout()
    return fig


def summarize_numeric_vs_target(
    df: pd.DataFrame,
    numeric_cols: list[str] | None = None,
    target_col: str = "exit_velo",
) -> pd.DataFrame:
    """
    Summarise each numeric predictor against the target.

    Returns a DataFrame indexed by feature with:
      n          – number of non‑null pairs
      pearson_r  – Pearson correlation coefficient
    """
    # --- Pull fresh lists from the schema every time -----------------
    groups = cols.as_dict()

    if numeric_cols is None:
        numeric_cols = groups.get("numerical", [])

    # --- Clean the list ---------------------------------------------
    numeric_cols = [
        c for c in numeric_cols
        if c != target_col and c in df.columns      # ❶ exclude target, ❷ guard
    ]

    records = []
    for col in numeric_cols:
        sub = df[[col, target_col]].dropna()
        if sub.empty:               # skip columns that are all‑NA
            continue
        r = sub[col].corr(sub[target_col])
        records.append({"feature": col, "n": len(sub), "pearson_r": r})

    result = (
        pd.DataFrame.from_records(records)
        .set_index("feature")
        .sort_values("pearson_r", ascending=False)
    )

    print("\n=== Numeric vs target correlations ===")
    print(result)

    return result


def plot_numeric_vs_target(
    df: pd.DataFrame,
    numeric_cols: list[str] | None = None,
    target_col: str = "exit_velo",
):
    """
    Scatter plots of each numeric predictor vs the target with r‑value in title.
    """
    summary = summarize_numeric_vs_target(df, numeric_cols, target_col)
    for feature, row in summary.iterrows():
        plt.figure(figsize=(6, 4))
        plt.scatter(
            df[feature], df[target_col],
            alpha=0.3, s=5, edgecolors="none"
        )
        plt.title(f"{feature} vs {target_col}  (r = {row['pearson_r']:.2f})")
        plt.xlabel(feature)
        plt.ylabel(target_col)
        plt.tight_layout()
        plt.show()



def summarize_categorical_vs_target(
    df: pd.DataFrame,
    cat_cols: list[str] | None = None,
    target_col: str = "exit_velo"
) -> dict[str, pd.DataFrame]:
    """
    For each categorical feature, returns a DataFrame of:
      count, mean, median, std of the target by category.
    """
    groups = get_column_groups()
    if cat_cols is None:
        cat_cols = groups.get("categorical", [])

    summaries: dict[str, pd.DataFrame] = {}
    for col in cat_cols:
        stats = (
            df
            .groupby(col)[target_col]
            .agg(count="count", mean="mean", median="median", std="std")
            .sort_values("count", ascending=False)
        )
        print(f"\n=== {col} vs {target_col} summary ===")
        print(stats)
        summaries[col] = stats
    return summaries


def plot_categorical_vs_target(
    df: pd.DataFrame,
    cat_cols: list[str] | None = None,
    target_col: str = "exit_velo"
):
    """
    For each categorical feature, draw a box‑plot of the target by category.
    """
    groups = get_column_groups()
    if cat_cols is None:
        cat_cols = groups.get("categorical", [])

    for col in cat_cols:
        plt.figure(figsize=(6, 4))
        df.boxplot(column=target_col, by=col, vert=False,
                   grid=False, patch_artist=True)
        plt.title(f"{target_col} by {col}")
        plt.suptitle("")           # remove pandas' automatic suptitle
        plt.xlabel(target_col)
        plt.tight_layout()
        plt.show()



def examine_and_filter_by_sample_size(
    df: pd.DataFrame,
    count_col: str = "exit_velo",
    group_col: str = "batter_id",
    season_col: str = "season",
    percentile: float = 0.05,
    min_count: int | None = None,
    filter_df: bool = False,
) -> tuple[dict[int, pd.DataFrame], pd.DataFrame | None]:
    """
    For each season:
      - compute per-batter count, mean, std of `count_col`
      - pick cutoff: min_count if provided, else the `percentile` quantile
      - print diagnostics
      - plot histograms *safely* (drops NaNs first)
    Returns:
      - summaries: dict season → per-batter summary DataFrame
      - filtered_df: if filter_df, the original df filtered to batters ≥ cutoff
    """
    summaries: dict[int, pd.DataFrame] = {}
    mask_keep: list[pd.Series] = []

    for season, sub in df.groupby(season_col):
        # 1) per-batter summary (count *non-NA* exit_velo)
        summary = (
            sub
            .groupby(group_col)[count_col]
            .agg(count="count", mean="mean", std="std")
            .sort_values("count")
        )
        summaries[season] = summary

        # 2) determine cutoff
        cutoff = min_count if min_count is not None else int(summary["count"].quantile(percentile))
        small = summary[summary["count"] < cutoff]
        large = summary[summary["count"] >= cutoff]

        # 3) diagnostics
        print(f"\n=== Season {season} (cutoff = {cutoff}) ===")
        print(f"  small (<{cutoff} events): {len(small)} batters")
        print(small[["count","mean","std"]].describe(), "\n")
        print(f"  large (≥{cutoff} events): {len(large)} batters")
        print(large[["count","mean","std"]].describe())

        # 4) **safe plotting**: drop NaNs, skip if nothing to plot
        small_means = small["mean"].dropna()
        large_means = large["mean"].dropna()

        if small_means.empty and large_means.empty:
            print(f"  ⚠️  Season {season}: no valid per-batter means to plot")
        else:
            plt.figure(figsize=(8, 3))
            if not small_means.empty:
                plt.hist(small_means, bins=30, alpha=0.6, label=f"n<{cutoff}")
            if not large_means.empty:
                plt.hist(large_means, bins=30, alpha=0.6, label=f"n≥{cutoff}")
            plt.title(f"Season {season}: per-batter EV means")
            plt.xlabel("Mean exit_velo")
            plt.legend()
            plt.tight_layout()
            plt.show()

        # 5) build mask to keep only large-sample batters
        if filter_df:
            keep_ids = large.index
            mask_keep.append(
                (df[season_col] == season) &
                (df[group_col].isin(keep_ids))
            )

    # 6) combine masks and filter
    filtered_df = None
    if filter_df and mask_keep:
        combined = pd.concat(mask_keep, axis=1).any(axis=1)
        filtered_df = df[combined].copy()

    return summaries, filtered_df



def hypothesis_test(df, feature, target="exit_velo", test_type="anova"):
    """
    Perform hypothesis tests for feature significance.
    """
    if test_type == "anova":
        groups = [df[df[feature] == cat][target] for cat in df[feature].unique()]
        F, p = f_oneway(*groups)
        print(f"ANOVA: F={F:.3f}, p={p:.3e}")
        return F, p
    elif test_type == "ttest":
        group1 = df[df[feature] == 0][target]
        group2 = df[df[feature] == 1][target]
        t, p = ttest_ind(group1, group2)
        print(f"T-test: t={t:.3f}, p={p:.3e}")
        return t, p


# ------------------------------------------------------------------
#  NEW: robust outlier flagging
# ------------------------------------------------------------------
def flag_outliers_iqr(
    df: pd.DataFrame,
    velo_col: str = "exit_velo",
    iqr_mult: float = 1.5,
) -> pd.Series:
    """
    🔹 Why it matters – extreme EVs (>120 mph or <40 mph) can distort
      skew / variance estimates used in hierarchical priors.

    Returns a boolean Series (True = *suspect* outlier) using the
    classic IQR rule: value < Q1 − k·IQR  or  > Q3 + k·IQR.
    """
    q1, q3 = df[velo_col].quantile([0.25, 0.75])
    iqr = q3 - q1
    lower, upper = q1 - iqr_mult * iqr, q3 + iqr_mult * iqr
    mask = (df[velo_col] < lower) | (df[velo_col] > upper)
    n = int(mask.sum())
    print(f"> Outlier flag ({velo_col}): {n} rows outside [{lower:.1f}, {upper:.1f}]")
    return mask



# ------------------------------------------------------------------
#  NEW: EV distribution summary + QQ plot
# ------------------------------------------------------------------
def ev_distribution_summary(
    df: pd.DataFrame,
    velo_col: str = "exit_velo",
    bins: int = 40,
):
    """
    🔹 Why it matters – confirms right‑skew & heavy‑tail nature of EV
      so you can choose a skew‑normal or Student‑t likelihood.

    Prints skew/kurtosis, shows histogram, KDE, CDF & QQ (if scipy).
    """
    data = df[velo_col].dropna()
    print(
        f"Skewness = {stats.skew(data):.2f},  "
        f"Kurtosis = {stats.kurtosis(data, fisher=False):.2f}"
    )
    fig, ax = plt.subplots(1, 3, figsize=(12, 3))
    ax[0].hist(data, bins=bins, density=True, alpha=0.7)
    data.plot(kind="kde", ax=ax[0], linewidth=2)
    ax[0].set_title("Histogram & KDE")

    # empirical CDF
    ecdf_x = np.sort(data)
    ecdf_y = np.arange(1, len(ecdf_x) + 1) / len(ecdf_x)
    ax[1].plot(ecdf_x, ecdf_y)
    ax[1].set_title("Empirical CDF")

    # QQ vs normal
    from scipy import stats as _st
    _st.probplot(data, dist="norm", plot=ax[2])
    ax[2].set_title("QQ‑plot vs Normal")
    plt.tight_layout()
    return fig


# ------------------------------------------------------------------
#  NEW: Year/era trend diagnostic
# ------------------------------------------------------------------
def year_trend_ev(
    df: pd.DataFrame,
    season_col: str = "season",
    velo_col: str = "exit_velo",
    ci: bool = True,
):
    """
    🔹 Why it matters – detects ball‑era shifts (e.g. 2019 “juiced”,
      2021 “deadened”) so forecasts for 2024 use correct baseline.

    Produces a table & line plot of mean/median EV per season.
    """
    g = df.groupby(season_col)[velo_col]
    stats_df = g.agg(mean="mean", median="median", n="count")
    print("\n=== Exit‑velo by season ===")
    print(stats_df)

    fig, ax = plt.subplots(figsize=(7, 3))
    stats_df["mean"].plot(ax=ax, marker="o", label="Mean EV")
    stats_df["median"].plot(ax=ax, marker="s", label="Median EV")
    if ci:
        sem = g.sem()
        ax.fill_between(
            stats_df.index,
            stats_df["mean"] - 1.96 * sem,
            stats_df["mean"] + 1.96 * sem,
            alpha=0.2,
            label="95% CI (mean)"
        )
    ax.set_ylabel(velo_col)
    ax.set_title("Seasonal trend in exit velocity")
    ax.legend()
    plt.tight_layout()
    return stats_df, fig



# ------------------------------------------------------------------
#  REPLACE old red_flag_level_effect  → clearer name & doc
# ------------------------------------------------------------------
def league_level_effect(
    df: pd.DataFrame,
    level_col: str = "level_abbr",
    velo_col: str = "exit_velo",
) -> tuple[float | None, float | None]:
    """
    🔹 Why it matters – confirms MLB vs Triple‑A (etc.) differences to
      justify hierarchical level effects in the model.

    One‑way ANOVA of `exit_velo` across `level_col`.
    Returns (F, p) or (None, None) if SciPy unavailable.
    """
    if not _HAS_STATS_LIBS:
        print("> SciPy unavailable – falling back to group summary")
        print(df.groupby(level_col)[velo_col].describe())
        return None, None

    groups = [df[df[level_col] == lv][velo_col].dropna()
              for lv in df[level_col].unique()]
    f_val, p_val = stats.f_oneway(*groups)
    print(f"> Level effect ANOVA: F={f_val:.3f}, p={p_val:.3e}")
    return f_val, p_val





# debugs:
def summarize_categorical_missingness(df: pd.DataFrame) -> pd.DataFrame:
    """
    For each categorical column (ordinal + nominal), compute:
      - original_null_count / pct
      - imputed_missing_count / pct
    Safely handles pandas.Categorical by first adding 'MISSING' to its categories.
    """
    cols    = _ColumnSchema()
    cat_cols = cols.ordinal() + cols.nominal()
    summary = []
    n = len(df)

    for col in cat_cols:
        ser = df[col]
        orig_null = ser.isna().sum()

        # If it's a Categorical, add 'MISSING' as a valid category
        if is_categorical_dtype(ser):
            ser = ser.cat.add_categories(['MISSING'])

        # Count rows that would become 'MISSING'
        imputed_missing = ser.fillna('MISSING').eq('MISSING').sum()

        summary.append({
            'column': col,
            'original_null_count':   orig_null,
            'original_null_pct':     orig_null / n,
            'imputed_missing_count': imputed_missing,
            'imputed_missing_pct':   imputed_missing / n,
        })

    return pd.DataFrame(summary)





if __name__ == "__main__":
    from pathlib import Path
    from src.data.load_data import load_and_clean_data
    from src.features.feature_engineering import feature_engineer

    raw_path = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    df = load_and_clean_data(raw_path)
    print(df.head())
    print(df.columns)

    # --- inspect nulls in the raw data ---
    null_counts = df.isnull().sum()
    null_counts = null_counts[null_counts > 0]
    if null_counts.empty:
        print("✅  No missing values in raw data.")
    else:
        print("=== Raw data null counts ===")
        for col, cnt in null_counts.items():
            print(f" • {col!r}: {cnt} missing")

    
    df_fe = feature_engineer(df)

    print("Raw →", df.shape, "//  Feature‑engineered →", df_fe.shape)
    print(df_fe.head())

    # singleton instance people can import as `cols`
    cols = _ColumnSchema()

    __all__ = ["cols"]
    print("ID columns:         ", cols.id())
    print("Ordinal columns:    ", cols.ordinal())
    print("Nominal columns:    ", cols.nominal())
    print("All categorical:    ", cols.categorical())
    print("Numerical columns:  ", cols.numerical())
    print("Model features:     ", cols.model_features())
    print("Target columns:     ", cols.target())
    print("All raw columns:    ", cols.all_raw())
    numericals = cols.numerical()
    # use list‐comprehension to drop target(s) from numerical features
    numericals_without_y = [c for c in numericals if c not in cols.target()]

    # Filter out rows with nulls in the target column(s)
    target_cols = cols.target()
    df_filtered = df.dropna(subset=target_cols)

    # Check for nulls in the filtered data
    null_counts_filtered = df_filtered.isnull().sum()
    null_counts_filtered = null_counts_filtered[null_counts_filtered > 0]
    if null_counts_filtered.empty:
        print("✅  No missing values in filtered data.")
    else:
        print("=== Filtered data null counts ===")
        for col, cnt in null_counts_filtered.items():
            print(f" • {col!r}: {cnt} missing")

    print("\n===== check on small samples =====")
    summaries, _ = examine_and_filter_by_sample_size(df_filtered, percentile=0.05)
    summaries, df_filtered = examine_and_filter_by_sample_size(
        df_filtered, percentile=0.05, min_count=15, filter_df=False
    )

    
    # Example usage
    print("\n===== NULLS CHECK =====")
    check_nulls(df_fe)
    
    print("\n===== QUICK PULSE CHECK =====")
    quick_pulse_check(df_fe)
    
    print("\n===== RED FLAGS CHECK =====")
    check_red_flags(df_fe)
    
    print("\n===== AGE EFFECT ANALYSIS =====")
    diag_age_effect(df_fe, age_col="age")
    
    print("\n===== TIME SERIES ANALYSIS =====")
    diag_time_series_dw(df_fe)
    
    print("\n===== PLOTTING =====")
    fig1 = plot_distributions(df_fe, by="hit_type")
    fig2 = plot_correlations(df_fe, numericals)  # Using cols schema
    fig3 = plot_time_trends(df_fe, sample=20)


    # — Numeric features —
    num_summary = summarize_numeric_vs_target(df_fe)
    plot_numeric_vs_target(df_fe)

    # — Categorical features —
    cat_summary = summarize_categorical_vs_target(df_fe)
    plot_categorical_vs_target(df_fe)

    # Example: Test if age has significant effect
    hypothesis_test(df_fe, feature="age_bin", test_type="anova")
    
    
    league_level_effect(df_fe)
    year_trend_ev(df_fe)
    flag_outliers_iqr(df_fe)
    ev_distribution_summary(df_fe)
    # _optional_dw_check(df_fe)   # only if you still care


    summary_df = summarize_categorical_missingness(df_fe)
    print(summary_df.to_markdown(index=False))
    
    # uniques of outcome
    print("outcome uniques:=========================")
    print(df_fe["outcome"].unique())
    # uniques of hit_type
    print("hit_type uniques:=========================")
    print(df_fe["hit_type"].unique())


    # assume df is your feature-engineered DataFrame
    df = df.copy()
    # 1) Flag missing hangtime
    df['hangtime_missing'] = df['hangtime'].isna()

    # 2) Contingency of hit_type vs missingness
    hit_type_ct = pd.crosstab(
        df['hit_type'].fillna('UNKNOWN'),
        df['hangtime_missing'],
        margins=True,
        normalize='columns'
    )
    print("\nProportion of hit_type when hangtime is missing:")
    print(hit_type_ct)

    # 3) Contingency of outcome vs missingness
    outcome_ct = pd.crosstab(
        df['outcome'].fillna('UNKNOWN'),
        df['hangtime_missing'],
        margins=True,
        normalize='columns'
    )
    print("\nProportion of outcome when hangtime is missing:")
    print(outcome_ct)


Overwriting src/features/eda.py


In [11]:
%%writefile src/features/data_prep.py
import pandas as pd
import numpy as np


# ─────────────────────────────────────────────────────────────────────────────
def compute_clip_bounds(
    series: pd.Series,
    *,
    method: str = "quantile",
    quantiles: tuple[float,float] = (0.01,0.99),
    std_multiplier: float = 3.0,
) -> tuple[float,float]:
    """
    Compute (lower, upper) but do not apply them.
    """
    s = series.dropna()
    if method == "quantile":
        return tuple(s.quantile(list(quantiles)).to_list())
    if method == "mean_std":
        mu, sigma = s.mean(), s.std()
        return mu - std_multiplier*sigma, mu + std_multiplier*sigma
    if method == "iqr":
        q1, q3 = s.quantile([0.25,0.75])
        iqr = q3 - q1
        return q1 - 1.5*iqr, q3 + 1.5*iqr
    raise ValueError(f"Unknown method {method}")


def clip_extreme_ev(
    df: pd.DataFrame,
    velo_col: str = "exit_velo",
    lower: float | None = None,
    upper: float | None = None,
    *,
    method: str = "quantile",
    quantiles: tuple[float, float] = (0.01, 0.99),
    std_multiplier: float = 3.0,
    debug: bool = False,
) -> pd.DataFrame:
    """
    Clip exit velocities to [lower, upper].

    If lower/upper are None, compute them from the data using one of:
      - method="quantile": use df[velo_col].quantile(quantiles)
      - method="mean_std":  use mean ± std_multiplier * std
      - method="iqr":       use [Q1 - 1.5*IQR, Q3 + 1.5*IQR]

    When debug=True, prints counts & percentages of values that will be clipped.
    """
    df = df.copy()
    series = df[velo_col].dropna()

    # 1) infer bounds if not given
    if lower is None or upper is None:
        if method == "quantile":
            low_q, high_q = quantiles
            lower_, upper_ = series.quantile([low_q, high_q]).to_list()
        elif method == "mean_std":
            mu, sigma = series.mean(), series.std()
            lower_, upper_ = mu - std_multiplier * sigma, mu + std_multiplier * sigma
        elif method == "iqr":
            q1, q3 = series.quantile([0.25, 0.75])
            iqr = q3 - q1
            lower_, upper_ = q1 - 1.5 * iqr, q3 + 1.5 * iqr
        else:
            raise ValueError(f"Unknown method '{method}' for clip_extreme_ev")
        lower  = lower  if lower  is not None else lower_
        upper  = upper  if upper  is not None else upper_

    # 2) debug: count how many will be clipped
    if debug:
        total   = len(series)
        n_low   = (series < lower).sum()
        n_high  = (series > upper).sum()
        print(f"[clip_extreme_ev] lower={lower:.2f}, upper={upper:.2f}")
        print(f"  → {n_low:,} / {total:,} ({n_low/total:.2%}) below lower")
        print(f"  → {n_high:,} / {total:,} ({n_high/total:.2%}) above upper")

    # 3) actually clip
    df[velo_col] = df[velo_col].clip(lower, upper)
    return df


# ─────────────────────────────────────────────────────────────────────────────
def filter_bunts_and_popups(
    df: pd.DataFrame,
    hit_col: str = "hit_type",
    debug: bool = False
) -> pd.DataFrame:
    """
    Drop bunts and pop-ups from the DataFrame.
    If debug=True, prints how many rows were removed.
    """
    df = df.copy()
    initial = len(df)
    mask = ~df[hit_col].str.upper().isin(["BUNT", "POP_UP"])
    filtered = df[mask]
    if debug:
        removed = initial - len(filtered)
        print(f"[filter_bunts_and_popups] dropped {removed:,} rows "
              f"({removed/initial:.2%}) bunts/pop-ups")
    return filtered


def filter_low_event_batters(
    df: pd.DataFrame,
    batter_col: str = "batter_id",
    min_events: int = 15,
    debug: bool = False
) -> pd.DataFrame:
    """
    Drop all rows for batters with fewer than min_events total events.
    """
    df = df.copy()
    counts = df[batter_col].value_counts()
    keep_batters = counts[counts >= min_events].index
    initial = len(df)
    filtered = df[df[batter_col].isin(keep_batters)]
    if debug:
        removed = initial - len(filtered)
        print(f"[filter_low_event_batters] dropped {removed:,} rows ({removed/initial:.2%}) for batters with <{min_events} events")
    return filtered


def filter_physical_implausibles(
    df: pd.DataFrame,
    hang_col: str = "hangtime",
    velo_col: str = "exit_velo",
    min_hang: float = 0.5,
    max_velo: float = 115.0,
    debug: bool = False
) -> pd.DataFrame:
    """
    Drop rows where hangtime < min_hang AND exit_velo > max_velo,
    as these are likely sensor glitches (e.g., foul tips).
    """
    df = df.copy()
    initial = len(df)
    mask = ~((df[hang_col] < min_hang) & (df[velo_col] > max_velo))
    filtered = df[mask]
    if debug:
        removed = initial - len(filtered)
        print(f"[filter_physical_implausibles] dropped {removed:,} rows ({removed/initial:.2%}) for hangtime<{min_hang} & exit_velo>{max_velo}")
    return filtered




# ─────────────────────────────────────────────────────────────────────────────
# Smoke test / CLI entry
# ─────────────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    from pathlib import Path
    from src.data.load_data import load_and_clean_data
    from src.data.ColumnSchema import _ColumnSchema
    from src.features.eda import summarize_categorical_missingness
    from src.features.feature_engineering import feature_engineer
    # from src.features.data_prep import filter_and_clip
    raw_path = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    df = load_and_clean_data(raw_path)
    print(df.head())
    print(df.columns)
    
    # singleton instance people can import as `cols`
    cols = _ColumnSchema()

    __all__ = ["cols"]
    print("ID columns:         ", cols.id())
    print("Ordinal columns:    ", cols.ordinal())
    print("Nominal columns:    ", cols.nominal())
    print("All categorical:    ", cols.categorical())
    print("Numerical columns:  ", cols.numerical())
    print("Model features:     ", cols.model_features())
    print("Target columns:  ", cols.target())
    print("All raw columns:    ", cols.all_raw())
    numericals = cols.numerical()
    # use list‐comprehension to drop target(s) from numerical features
    numericals_without_y = [c for c in numericals if c not in cols.target()]
    

    df_fe = feature_engineer(df)

    summary_df = summarize_categorical_missingness(df_fe)
    print(summary_df.to_markdown(index=False))
    
    print("Raw →", df.shape, "//  Feature‑engineered →", df_fe.shape)
    print(df_fe.head())
    
    # filter out bunts and popups
    df_fe = filter_bunts_and_popups(df_fe)
    # chekc on bunts and popups
    print(df_fe["hit_type"].unique())
    print(df_fe["outcome"].unique())

    debug = True
    TARGET = cols.target()
    lower, upper = compute_clip_bounds(
        df[TARGET],
        method="quantile",            # default: 1st/99th percentile
        quantiles=(0.01, 0.99),
    )
    if debug:
        total = len(df)
        n_low = (df[TARGET] < lower).sum()
        n_high= (df[TARGET] > upper).sum()
        print(f"[fit_preprocessor] clipping train EV to [{lower:.2f}, {upper:.2f}]")
        print(f"  → {n_low:,}/{total:,} ({n_low/total:.2%}) below")
        print(f"  → {n_high:,}/{total:,} ({n_high/total:.2%}) above")
    df_clipped = clip_extreme_ev(df, lower=lower, upper=upper)

    print("Final rows after filter & clip:", len(df_clipped))


    # 1) drop batters with too few events
    df_fe = filter_low_event_batters(df_fe, batter_col="batter_id", min_events=15, debug=debug)

    # 2) drop physical implausibles
    df_fe = filter_physical_implausibles(
        df_fe,
        hang_col="hangtime",
        velo_col="exit_velo",
        debug=debug)

Overwriting src/features/data_prep.py


# Preprocessing

In [12]:
%%writefile src/features/preprocess.py
"""
Preprocessing module for exit velocity pipeline.
Supports multiple model types (linear, XGBoost, PyMC, etc.) with
automatic ordinal-category detection from the data.
"""
import pandas as pd
import numpy as np
import warnings
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import SimpleImputer, IterativeImputer, MissingIndicator
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

from src.data.ColumnSchema import _ColumnSchema
from sklearn.model_selection import train_test_split
from src.features.data_prep import (filter_bunts_and_popups
                                    , compute_clip_bounds
                                    , clip_extreme_ev
                                    , filter_low_event_batters
                                    , filter_physical_implausibles)


# ───────────────────────────────────────────────────────────────────────
# Numeric & nominal pipelines (unchanged)
# ───────────────────────────────────────────────────────────────────────
numeric_linear = Pipeline([
    ('impute', SimpleImputer(strategy='median', add_indicator=True)),
    ('scale', StandardScaler()),
])
numeric_iterative = Pipeline([
    ('impute', IterativeImputer(random_state=0, add_indicator=True)),
    ('scale', StandardScaler()),
])
nominal_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='constant', fill_value='MISSING')),
    ('encode', OneHotEncoder(drop='first', handle_unknown='ignore')),
])

# ───────────────────────────────────────────────────────────────────────
# Helper function for domain cleaning to reduce duplication
# ───────────────────────────────────────────────────────────────────────
def filter_and_clip(
    df: pd.DataFrame,
    lower: float = None,
    upper: float = None,
    quantiles: tuple[float, float] = (0.01, 0.99),
    debug: bool = False
) -> pd.DataFrame:
    """
    Clean the dataset by:
    1. Filtering out bunts and popups
    2. Clipping extreme exit velocity values
    
    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame to clean
    lower : float, optional
        Lower bound for clipping, computed from data if None
    upper : float, optional
        Upper bound for clipping, computed from data if None
    quantiles : tuple[float, float], optional
        Quantiles to use if computing bounds from data
    debug : bool, optional
        Whether to print diagnostic information
        
    Returns
    -------
    pd.DataFrame
        Cleaned DataFrame (copy of input)
    tuple[float, float]
        The (lower, upper) bounds used for clipping
    """
    cols = _ColumnSchema()
    TARGET = cols.target()
    
    # 1. Filter out bunts and popups
    df = filter_bunts_and_popups(df, debug=debug)
    
    # 2. Compute bounds if not provided
    if lower is None or upper is None:
        lower_computed, upper_computed = compute_clip_bounds(
            df[TARGET], method="quantile", quantiles=quantiles
        )
        lower = lower if lower is not None else lower_computed
        upper = upper if upper is not None else upper_computed
    
    # 3. Clip extreme values
    df = clip_extreme_ev(df, lower=lower, upper=upper, debug=debug)

    # 1) drop batters with too few events
    df = filter_low_event_batters(df, batter_col="batter_id", min_events=15, debug=debug)

    # 2) drop physical implausibles
    df = filter_physical_implausibles(
        df,
        hang_col="hangtime",
        velo_col="exit_velo",
        debug=debug)
    
    return df, (lower, upper)

# ───────────────────────────────────────────────────────────────────────
# Dynamic preprocess functions
# ───────────────────────────────────────────────────────────────────────
# ─────────────────────────────────────────────────────────────────────────────
# 1) fit_preprocessor (with train‐set‐only bound computation + clip)  
# ─────────────────────────────────────────────────────────────────────────────
def fit_preprocessor(
    df: pd.DataFrame,
    model_type: str = "linear",
    debug: bool = False,
    quantiles: tuple[float, float] = (0.01, 0.99),
    max_safe_rows: int = 200000
) -> tuple[np.ndarray, pd.Series, ColumnTransformer]:
    """
    Fit a preprocessing pipeline to transform raw data into model-ready features,
    adding a MissingIndicator for ordinal features.

    IMPORTANT: Call this on TRAINING data only.
    """
    if len(df) > max_safe_rows:
        warnings.warn(
            f"Dataset has {len(df)} rows (>{max_safe_rows}); "
            "ensure you’re only fitting on TRAINING data.",
            UserWarning
        )

    # 0) Domain cleaning and clipping
    df, (lower, upper) = filter_and_clip(df, quantiles=quantiles, debug=debug)

    cols = _ColumnSchema()
    TARGET = cols.target()

    # 1) Split out features + coerce numerics
    num_feats = [c for c in cols.numerical() if c != TARGET]
    ord_feats = cols.ordinal()
    nom_feats = cols.nominal()

    df[num_feats] = df[num_feats].apply(pd.to_numeric, errors="coerce")
    X = df[num_feats + ord_feats + nom_feats].copy()
    y = df[TARGET]

    # 2) Prepare ordinal columns
    X[ord_feats] = X[ord_feats].astype("string")
    X.loc[:, ord_feats] = X.loc[:, ord_feats].mask(X[ord_feats].isna(), np.nan)
    ordinal_categories = [[*X[c].dropna().unique(), "MISSING"] for c in ord_feats]
    ordinal_pipe = Pipeline([
        ("impute", SimpleImputer(strategy="constant", fill_value="MISSING")),
        ("encode", OrdinalEncoder(
            categories=ordinal_categories,
            handle_unknown="use_encoded_value",
            unknown_value=-1,
            dtype="int32"
        )),
    ])

    # 3) Numeric pipeline selection
    if model_type == "linear":
        num_imputer = SimpleImputer(strategy="median", add_indicator=True)
    else:
        num_imputer = IterativeImputer(
            random_state=0,
            add_indicator=True
        )

    numeric_pipe = Pipeline([
        ("impute", num_imputer),
        ("scale", StandardScaler()),
    ])

    # 4) ColumnTransformer with an extra MissingIndicator for ordinals
    ct = ColumnTransformer(
        [
            ("num", numeric_pipe, num_feats),
            ("ord_ind", MissingIndicator(missing_values=np.nan), ord_feats),
            ("ord", ordinal_pipe, ord_feats),
            ("nom", nominal_pipe, nom_feats),
        ],
        remainder="drop",
        verbose_feature_names_out=False,
    )
    # preserve clipping bounds for transform
    ct.lower_, ct.upper_ = lower, upper

    X_mat = ct.fit_transform(X, y)
    return X_mat, y, ct




# ─────────────────────────────────────────────────────────────────────────────
def transform_preprocessor(
    df: pd.DataFrame,
    transformer: ColumnTransformer,
) -> tuple[np.ndarray, pd.Series]:
    """
    Transform new data using a fitted preprocessor.
    
    Parameters
    ----------
    df : pd.DataFrame
        New data to transform
    transformer : ColumnTransformer
        Fitted transformer from fit_preprocessor
        
    Returns
    -------
    X_mat : np.ndarray
        Transformed feature matrix
    y : pd.Series
        Target values
    """
    cols, TARGET = _ColumnSchema(), _ColumnSchema().target()

    # Domain filter & clip using the bounds from the fitted transformer
    df, _ = filter_and_clip(df, lower=transformer.lower_, upper=transformer.upper_)

    # rebuild lists & coerce numerics
    num_feats = [c for c in cols.numerical() if c != TARGET]
    ord_feats = cols.ordinal()
    nom_feats = cols.nominal()
    df[num_feats] = df[num_feats].apply(pd.to_numeric, errors="coerce")

    X = df[num_feats + ord_feats + nom_feats].copy()

    # **NEW** – ensure ordinals are strings, then fill unseen→"MISSING"
    X[ord_feats] = X[ord_feats].astype("string")
    X.loc[:, ord_feats] = X.loc[:, ord_feats].where(X.loc[:, ord_feats].notna(), "MISSING")

    y = df[TARGET]
    X_mat = transformer.transform(X)
    return X_mat, y


def inverse_transform_preprocessor(
    X_trans: np.ndarray,
    transformer: ColumnTransformer
) -> pd.DataFrame:
    """
    Invert each block of a ColumnTransformer back to its original features,
    skipping transformers that lack inverse_transform (e.g., MissingIndicator).
    """
    import numpy as np
    import pandas as pd
    import warnings
    from sklearn.impute import MissingIndicator  # for isinstance check

    # 1) Recover original feature order
    orig_features: list[str] = []
    for name, _, cols in transformer.transformers_:
        if cols == 'drop': continue
        orig_features.extend(cols)

    parts = []
    start = 0
    n_rows = X_trans.shape[0]

    # 2) Loop through each transformer block
    for name, trans, cols in transformer.transformers_:
        if cols == 'drop':
            continue

        fitted = transformer.named_transformers_[name]

        # 2a) Determine number of output columns
        dummy = np.zeros((1, len(cols)))
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=UserWarning)
            try:
                out = fitted.transform(dummy)
            except Exception:
                out = dummy
        n_out = out.shape[1]

        # 2b) Slice the real transformed data block
        block = X_trans[:, start : start + n_out]
        start += n_out

        # 2c) Handle each transformer type
        if isinstance(fitted, MissingIndicator):
            # Skip MissingIndicator: it has no inverse_transform
            continue  # :contentReference[oaicite:2]{index=2}
        elif trans == 'passthrough':
            inv = block
        elif name == 'num':
            scaler = fitted.named_steps['scale']
            inv_full = scaler.inverse_transform(block)
            inv = inv_full[:, :len(cols)]
        else:
            # Ordinal or nominal pipelines
            if isinstance(fitted, Pipeline):
                last = list(fitted.named_steps.values())[-1]
                inv = last.inverse_transform(block)
            else:
                inv = fitted.inverse_transform(block)

        parts.append(pd.DataFrame(inv, columns=cols, index=range(n_rows)))

    # 3) Concatenate & reorder to match original features
    df_orig = pd.concat(parts, axis=1)
    return df_orig[orig_features]





def prepare_for_mixed_and_hierarchical(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans the rows *and* adds convenience covariates expected by the
    hierarchical and mixed-effects models.
    """
    cols = _ColumnSchema()
    TARGET = cols.target()

    # Apply domain cleaning with default quantiles
    df, _ = filter_and_clip(df.copy())

    # Category coding for batter
    df["batter_id"] = df["batter_id"].astype("category")

    # New: Category coding for season
    df["season_cat"] = df["season"].astype("category")
    df["season_idx"] = df["season_cat"].cat.codes

    # New: Category coding for pitcher
    df["pitcher_cat"] = df["pitcher_id"].astype("category")
    df["pitcher_idx"] = df["pitcher_cat"].cat.codes

    return df











# ───────────────────────────────────────────────────────────────────────
# 6. Smoke test (only run when module executed directly)
# ───────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    from pathlib import Path
    from src.data.load_data import load_and_clean_data
    from src.features.feature_engineering import feature_engineer
    raw_path = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    df = load_and_clean_data(raw_path)
    print(df.head())
    print(df.columns)
    df_fe = feature_engineer(df)
    print("Raw →", df.shape, "//  Feature‑engineered →", df_fe.shape)
    print(df_fe.head())
    # singleton instance people can import as `cols`
    cols = _ColumnSchema()

    __all__ = ["cols"]
    print("ID columns:         ", cols.id())
    print("Ordinal columns:    ", cols.ordinal())
    print("Nominal columns:    ", cols.nominal())
    print("All categorical:    ", cols.categorical())
    print("Numerical columns:  ", cols.numerical())
    print("Model features:     ", cols.model_features())
    print("Target columns:  ", cols.target())
    print("All raw columns:    ", cols.all_raw())
    numericals = cols.numerical()


    train_df, test_df = train_test_split(df_fe, test_size=0.2, random_state=42)
    # run with debug prints
    X_train, y_train, tf = fit_preprocessor(train_df, model_type='linear', debug=True)
    X_test,  y_test      = transform_preprocessor(test_df, tf)

    print("Processed shapes:", X_train.shape, X_test.shape)

    # Example of inverse transform: 
    print("==========Example of inverse transform:==========")
    df_back = inverse_transform_preprocessor(X_train, tf)
    print("\n✅ Inverse‐transformed head (should mirror your original X_train):")
    print(df_back.head())
    print("Shape:", df_back.shape, "→ original X_train shape before transform:", X_train.shape)





Overwriting src/features/preprocess.py


# feature selection

In [13]:
%%writefile src/features/feature_selection.py
import pandas as pd

# ── NEW: model and importance imports
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
import shap

from pathlib import Path
from src.data.load_data import load_and_clean_data
from src.features.feature_engineering import feature_engineer
from src.data.ColumnSchema import _ColumnSchema
# ── NEW: shapash and shapiq imports
from shapash import SmartExplainer
import shapiq
from sklearn.utils import resample

def train_baseline_model(X, y):
    """
    Fit a RandomForestRegressor on X, y.
    Returns the fitted model.
    """
    # You can adjust hyperparameters as needed
    model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    model.fit(X, y)
    return model



def compute_permutation_importance(
    model,
    X: pd.DataFrame,
    y: pd.Series,
    n_repeats: int = 10,
    n_jobs: int = 1,
    max_samples: float | int = None,
    random_state: int = 42,
    verbose: bool = True
) -> pd.DataFrame:
    """
    Compute permutation importances with controlled resource usage.
    
    Parameters
    ----------
    model : estimator
        Fitted model implementing .predict and .score.
    X : pd.DataFrame
        Features.
    y : pd.Series or array
        Target.
    n_repeats : int
        Number of shuffles per feature.
    n_jobs : int
        Number of parallel jobs (avoid -1 on Windows).
    max_samples : float or int, optional
        If float in (0,1], fraction of rows to sample.
        If int, absolute number of rows to sample.
    random_state : int
        Seed for reproducibility.
    verbose : bool
        Print debug info if True.
        
    Returns
    -------
    pd.DataFrame
        Columns: feature, importance_mean, importance_std.
        Sorted descending by importance_mean.
    """
    # Debug info
    if verbose:
        print(f"⏳ Computing permutation importances on {X.shape[0]} rows × {X.shape[1]} features")
        print(f"   n_repeats={n_repeats}, n_jobs={n_jobs}, max_samples={max_samples}")

    # Subsample if requested
    X_sel, y_sel = X, y
    if max_samples is not None:
        if isinstance(max_samples, float):
            nsamp = int(len(X) * max_samples)
        else:
            nsamp = int(max_samples)
        if verbose:
            print(f"   Subsampling to {nsamp} rows for speed")
        X_sel, y_sel = resample(X, y, replace=False, n_samples=nsamp, random_state=random_state)

    try:
        result = permutation_importance(
            model,
            X_sel, y_sel,
            n_repeats=n_repeats,
            random_state=random_state,
            n_jobs=n_jobs,
        )
    except OSError as e:
        # Graceful fallback to single job
        if verbose:
            print(f"⚠️  OSError ({e}). Retrying with n_jobs=1")
        result = permutation_importance(
            model,
            X_sel, y_sel,
            n_repeats=n_repeats,
            random_state=random_state,
            n_jobs=1,
        )

    # Build and sort DataFrame
    importance_df = (
        pd.DataFrame({
            "feature": X.columns,
            "importance_mean": result.importances_mean,
            "importance_std": result.importances_std,
        })
        .sort_values("importance_mean", ascending=False)
        .reset_index(drop=True)
    )
    if verbose:
        print("✅ Permutation importances computed.")
    return importance_df


def compute_shap_importance(model, X, nsamples=100):
    """
    Compute mean absolute SHAP values per feature.
    Returns a DataFrame sorted by importance.
    """
    # DeepExplainer or TreeExplainer for tree-based models
    explainer = shap.TreeExplainer(model)
    # sample for speed
    X_sample = X.sample(n=min(nsamples, len(X)), random_state=42)
    shap_values = explainer.shap_values(X_sample)
    # For regression, shap_values is a 2D array
    mean_abs_shap = pd.DataFrame({
        "feature": X_sample.columns,
        "shap_importance": np.abs(shap_values).mean(axis=0),
    })
    mean_abs_shap = mean_abs_shap.sort_values("shap_importance", ascending=False).reset_index(drop=True)
    return mean_abs_shap



def filter_permutation_features(
    importance_df: pd.DataFrame,
    threshold: float
) -> list[str]:
    """
    Return features whose permutation importance_mean exceeds threshold.
    """
    kept = importance_df.loc[
        importance_df["importance_mean"] > threshold, "feature"
    ]
    return kept.tolist()


def filter_shap_features(
    importance_df: pd.DataFrame,
    threshold: float
) -> list[str]:
    """
    Return features whose shap_importance exceeds threshold.
    """
    kept = importance_df.loc[
        importance_df["shap_importance"] > threshold, "feature"
    ]
    return kept.tolist()


def select_final_features(
    perm_feats: list[str],
    shap_feats: list[str],
    mode: str = "intersection"
) -> list[str]:
    """
    Combine permutation and SHAP feature lists.
    mode="intersection" for features in both lists,
    mode="union" for features in either list.
    """
    set_perm = set(perm_feats)
    set_shap = set(shap_feats)
    if mode == "union":
        final = set_perm | set_shap
    else:
        final = set_perm & set_shap
    # return sorted for reproducibility
    return sorted(final)



def load_final_features(
    file_path: str = "data/models/features/final_features.txt"
) -> list[str]:
    """
    Read the newline-delimited feature names file and return as a list.
    """
    with open(file_path, "r") as fp:
        return [line.strip() for line in fp if line.strip()]


def filter_to_final_features(
    df: pd.DataFrame,
    file_path: str = "data/models/features/final_features.txt"
) -> pd.DataFrame:
    """
    Given a feature-engineered DataFrame, load the final feature list,
    then return df[ ID_cols + final_features + [target] ].
    """
    # load the feature names
    final_feats = load_final_features(file_path)
    cols = _ColumnSchema()

    keep = cols.id() + final_feats + [cols.target()]
    missing = set(keep) - set(df.columns)
    if missing:
        raise ValueError(f"Cannot filter: missing columns {missing}")
    return df[keep].copy()





if __name__ == "__main__":
    # --- existing loading & schema logic ---
    raw_path = Path("data/Research Data Project/Research Data Project/exit_velo_project_data.csv")
    df = load_and_clean_data(raw_path)
    print(df.head())
    print(df.columns)
    null_counts = df.isnull().sum()
    null_counts = null_counts[null_counts > 0]
    if null_counts.empty:
        print("✅  No missing values in raw data.")
    else:
        print("=== Raw data null counts ===")
        for col, cnt in null_counts.items():
            print(f" • {col!r}: {cnt} missing")
    df_fe = feature_engineer(df)
    print("Raw →", df.shape, "//  Feature-engineered →", df_fe.shape)
    print(df_fe.head())

    cols = _ColumnSchema()
    print("ID columns:         ", cols.id())
    print("Ordinal columns:    ", cols.ordinal())
    print("Nominal columns:    ", cols.nominal())
    print("All categorical:    ", cols.categorical())
    print("Numerical columns:  ", cols.numerical())
    print("Model features:     ", cols.model_features())
    print("Target columns:     ", cols.target())
    print("All raw columns:    ", cols.all_raw())
    numericals = cols.numerical()
    numericals_without_y = [c for c in numericals if c not in cols.target()]

    # ── STEP 1: fully preprocess the engineered DataFrame ──
    from src.features.preprocess import fit_preprocessor, inverse_transform_preprocessor

    # fit_preprocessor returns (X_matrix, y, fitted_transformer)
    X_np, y, preproc = fit_preprocessor(df_fe, model_type='linear', debug=False)

    # Use the same index that y carries (only non-bunt, non-NA rows)
    idx = y.index
    
    # turn that into a DataFrame with the same column names:
    feat_names = preproc.get_feature_names_out()
    X = pd.DataFrame(X_np, columns=feat_names, index=idx)
    print(f"✅ Preprocessed feature matrix: {X.shape[0]} rows × {X.shape[1]} cols")

    # (optional) confirm inverse transform lines up:
    df_back = inverse_transform_preprocessor(X_np, preproc)
    df_back.index = idx
    print("✅ inverse_transform round-trip (head):")
    print(df_back.head())

    # ── STEP 2: train & compute importances on *that* X ──
    print("\nTraining baseline model…")
    model = train_baseline_model(X, y)

    print("\n🔍 Permutation Importances:")
    perm_imp = compute_permutation_importance(
        model, X, y,
        n_repeats=10,
        n_jobs=2,            # test small parallelism
        max_samples=0.5,     # test subsampling
        verbose=True
    )
    print(perm_imp)


    print("\n🔍 SHAP Importances:")
    shap_imp = compute_shap_importance(model, X)
    print(shap_imp)

    # ── STEP 3: threshold & select your final features ──
    perm_thresh = 0.01
    shap_thresh = 0.01
    perm_feats = filter_permutation_features(perm_imp, perm_thresh)
    shap_feats = filter_shap_features(shap_imp, shap_thresh)
    final_feats = select_final_features(perm_feats, shap_feats, mode="intersection")
    print(f"\nFinal preprocessed feature list ({len(final_feats)}):")
    print(final_feats)

    # ── STEP 4: build & save a dataset with just those features + target + IDs ──
    df_final = pd.concat([
        df_fe[cols.id()],
        df_fe[[cols.target()]],
        X[final_feats]
    ], axis=1)
    print("Final dataset shape:", df_final.shape)

    Path("data/models/features/final_features.txt").write_text("\n".join(final_feats))
    print("✔️ Saved feature list to final_features.txt")


    # Demo: filter the full df_fe back to just those features
    df_filtered = filter_to_final_features(df_fe)
    print("Filtered to final features shape:", df_filtered.shape)


Overwriting src/features/feature_selection.py


# model choices

see modelling_choices.txt

In [14]:
%%writefile src/models/linear.py

"""
Fast linear baselines (OLS and Ridge).

Usage
-----
>>> from src.models.linear import fit_ridge
>>> fitted, rmse = fit_ridge(train_df, val_df)
"""
from __future__ import annotations
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline


def _split_xy(df: pd.DataFrame):
    X = df.drop(columns=["exit_velo"])
    y = df["exit_velo"]
    return X, y


def fit_ridge(X_tr: pd.DataFrame,
              y_tr: pd.DataFrame,
              X_te: pd.DataFrame,
              y_te: pd.DataFrame,
              alpha: float = 1.0):
    """
    Returns (sklearn Pipeline, RMSE on test set).
    """

    model = Pipeline(
        [("reg" , Ridge(alpha=alpha, random_state=0))]
    ).fit(X_tr, y_tr)

    pred = model.predict(X_te)
    rmse = np.sqrt(np.mean((pred - y_te) ** 2))
    return model, rmse



if __name__ == "__main__":
    from pathlib import Path
    from src.data.load_data import load_and_clean_data
    from src.features.feature_engineering import feature_engineer
    from src.data.ColumnSchema import _ColumnSchema
    from sklearn.model_selection import train_test_split
    from src.features.preprocess import summarize_categorical_missingness
    from src.features.preprocess import fit_preprocessor, transform_preprocessor, inverse_transform_preprocessor
    raw_path = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    df = load_and_clean_data(raw_path)
    print(df.head())
    print(df.columns)

    # --- inspect nulls in the raw data ---
    null_counts = df.isnull().sum()
    null_counts = null_counts[null_counts > 0]
    if null_counts.empty:
        print("✅  No missing values in raw data.")
    else:
        print("=== Raw data null counts ===")
        for col, cnt in null_counts.items():
            print(f" • {col!r}: {cnt} missing")
    df_fe = feature_engineer(df)

    print("Raw →", df.shape, "//  Feature‑engineered →", df_fe.shape)
    print(df_fe.head())

    # singleton instance people can import as `cols`
    cols = _ColumnSchema()

    __all__ = ["cols"]
    print("ID columns:         ", cols.id())
    print("Ordinal columns:    ", cols.ordinal())
    print("Nominal columns:    ", cols.nominal())
    print("All categorical:    ", cols.categorical())
    print("Numerical columns:  ", cols.numerical())
    print("Model features:     ", cols.model_features())
    print("Target columns:  ", cols.target())
    print("All raw columns:    ", cols.all_raw())
    numericals = cols.numerical()
    # use list‐comprehension to drop target(s) from numerical features
    numericals_without_y = [c for c in numericals if c not in cols.target()]

    summary_df = summarize_categorical_missingness(df_fe)
    print(summary_df.to_markdown(index=False))


    # check nulls
    print("🛠️  Nulls in X before fit_transform:")
    null_counts = df_fe.isnull().sum()
    null_counts = null_counts[null_counts > 0]
    if null_counts.empty:
        print("✅  No missing values after feature engineering.")
    else:
        print("=== Null counts post-engineering ===")
        print(null_counts)

    train_df, test_df = train_test_split(df_fe, test_size=0.2, random_state=42)
    
    # only on training data for linear/XGB
    train_df = clip_extreme_ev(train_df)
    #valid_df = clip_extreme_ev(valid_df)
    
    # run with debug prints
    X_train, y_train, tf = fit_preprocessor(train_df, model_type='linear', debug=True)
    X_test,  y_test      = transform_preprocessor(test_df, tf)

        
    print("Processed shapes:", X_train.shape, X_test.shape)

    # Example of inverse transform: 
    print("==========Example of inverse transform:==========")
    df_back = inverse_transform_preprocessor(X_train, tf)
    print("\n✅ Inverse‐transformed head (should mirror your original X_train):")
    print(df_back.head())
    print("Shape:", df_back.shape, "→ original X_train shape before transform:", X_train.shape)
    

    # === NEW: Train & evaluate Ridge regression ===
    model_ridge, rmse_ridge = fit_ridge(X_train, y_train, X_test,  y_test)
    print(f"Ridge regression RMSE: {rmse_ridge:.4f}")


Overwriting src/models/linear.py


In [15]:
%%writefile src/utils/gbm_utils.py

import joblib
from pathlib import Path

def save_pipeline(model, preprocessor, path: str = "data/models/saved_models/gbm_pipeline.joblib"):
    """
    Save both model and preprocessor together to a single file.
    """
    out_path = Path(path)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    joblib.dump(
        {"model": model, "preprocessor": preprocessor},
        out_path
    )
    print(f"✅ Pipeline saved to {out_path.resolve()}")

def load_pipeline(path: str = "data/models/saved_models/gbm_pipeline.joblib"):
    """
    Load the model+preprocessor dict back into memory.
    Returns a tuple: (model, preprocessor)
    """
    saved = joblib.load(path)
    return saved["model"], saved["preprocessor"]


Overwriting src/utils/gbm_utils.py


In [16]:
%%writefile src/models/gbm.py

"""
Gradient‑boosting baseline (XGBoost regressor).
"""
from __future__ import annotations
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from xgboost.callback import EarlyStopping     # ① import the new callback
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
import optuna
from src.data.ColumnSchema import _ColumnSchema
from xgboost.core import XGBoostError
from src.utils.gbm_utils import save_pipeline, load_pipeline

# ————— Detect GPU support using modern API —————
try:
    XGBRegressor(tree_method="hist", device="cuda")
    GPU_SUPPORT = True
except XGBoostError:
    GPU_SUPPORT = False

def _split_xy(df: pd.DataFrame):
    X = df.drop(columns=["exit_velo"])
    y = df["exit_velo"]
    return X, y

def tune_gbm(X, y, n_trials: int = 50):
    """
    Run an Optuna study to minimize CV RMSE of an XGBRegressor.
    Uses device=cuda if available, else CPU.
    """
    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "random_state": 0,
            "n_jobs": -1,
        }
        params.update({"tree_method": "hist"})
        model = XGBRegressor(**params)
        scores = cross_val_score(
            model, X, y,
            scoring="neg_root_mean_squared_error",
            cv=3, n_jobs=-1
        )
        return -scores.mean()

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=n_trials)
    return study.best_trial.params



def fit_gbm(X_tr, y_tr, X_te, y_te, **gbm_kw):
    """
    Train an XGBRegressor with optional hyperparameters and early stopping.
    Returns (fitted_model, rmse_on_test).
    """

    # A) Default constructor args
    constructor_defaults = dict(
        n_estimators=600,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.7,
        colsample_bytree=0.6,
        random_state=0,
        n_jobs=-1,
        tree_method="hist",
    )
    constructor_defaults.update(gbm_kw)

    # B) Extract early_stopping_rounds (pop it out)
    early_stopping = constructor_defaults.pop("early_stopping_rounds", None)

    # C) Instantiate model
    model = XGBRegressor(**constructor_defaults)

    # D) Prepare fit kwargs
    fit_kwargs = {}
    # If an early_stopping value was provided, use the callback interface
    if early_stopping:
        fit_kwargs["callbacks"] = [EarlyStopping(rounds=early_stopping)]
        # pass eval_set so callback can work
        fit_kwargs["eval_set"] = [(X_te, y_te)]
    # You can still pass verbose if desired
    fit_kwargs["verbose"] = False

    # Fit the model with or without early stopping
    model.fit(X_tr, y_tr, **fit_kwargs)

    # E) Compute RMSE on test
    preds = model.predict(X_te)
    rmse  = mean_squared_error(y_te, preds, squared=False)

    return model, rmse





# ───────────────────────────────────────────────────────────────────────
# 6. Smoke test (only run when module executed directly)
# ───────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    from pathlib import Path
    from src.data.load_data import load_and_clean_data
    from src.features.feature_engineering import feature_engineer
    from src.features.preprocess import (fit_preprocessor
                                        ,transform_preprocessor
                                        ,inverse_transform_preprocessor)
    raw_path = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    df = load_and_clean_data(raw_path)
    print(df.head())
    print(df.columns)
    df_fe = feature_engineer(df)
    print("Raw →", df.shape, "//  Feature‑engineered →", df_fe.shape)
    print(df_fe.head())
    # singleton instance people can import as `cols`
    cols = _ColumnSchema()

    __all__ = ["cols"]
    print("ID columns:         ", cols.id())
    print("Ordinal columns:    ", cols.ordinal())
    print("Nominal columns:    ", cols.nominal())
    print("All categorical:    ", cols.categorical())
    print("Numerical columns:  ", cols.numerical())
    print("Model features:     ", cols.model_features())
    print("Target columns:  ", cols.target())
    print("All raw columns:    ", cols.all_raw())
    numericals = cols.numerical()


    train_df, test_df = train_test_split(df_fe, test_size=0.2, random_state=42)
    # run with debug prints
    # Record original feature count before preprocessing
    X_orig = train_df.drop(columns=["exit_velo"])
    orig_shape = X_orig.shape

    # Run preprocessing
    X_train, y_train, tf = fit_preprocessor(train_df, model_type='linear', debug=True)
    X_test,  y_test      = transform_preprocessor(test_df, tf)

    print("Processed shapes:", X_train.shape, X_test.shape)

    # Example of inverse transform: 
    print("==========Example of inverse transform:==========")
    df_back = inverse_transform_preprocessor(X_train, tf)
    print("\n✅ Inverse‐transformed head (should mirror your original X_train):")
    print(df_back.head())
    print(f"Shape: {df_back.shape} → original X_train shape before transform: {orig_shape}")


        

    # === Hyperparameter tuning ===
    best_params = tune_gbm(X_train, y_train, n_trials=5)
    print("Tuned params:", best_params)

    # === Train & evaluate ===
    gbm_model, rmse = fit_gbm(
        X_train, y_train, X_test, y_test, **best_params
    )
    print(f"Tuned XGBoost RMSE: {rmse:.4f}")


    # 3) save both at once
    save_pipeline(gbm_model, tf, path="models/gbm_pipeline.joblib")
    
    model, preprocessor = load_pipeline("models/gbm_pipeline.joblib")
    
    

Overwriting src/models/gbm.py


In [17]:
%%writefile src/models/mixed.py

"""
Frequentist mixed‑effects model using statsmodels MixedLM.

Formula implemented:
    exit_velo ~ 1 + level_ord + age_centered
              + (1 | batter_id)

We rely on columns already produced by preprocess():
    • level_idx  (0,1,2)   – ordinal
    • age_centered
"""
from __future__ import annotations
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf


def fit_mixed(train: pd.DataFrame,
              test: pd.DataFrame):
    """Return (fitted model, RMSE on test)."""
    # statsmodels wants a *single* DataFrame with all cols
    # so we concatenates and keep row positions for slicing
    combined = pd.concat([train, test], axis=0)
    # ensure categorical dtype
    combined["level_ord"] = combined["level_idx"].astype(int)

    mdl = smf.mixedlm(
        formula="exit_velo ~ 1 + level_ord + age_centered",
        data=combined.iloc[: len(train)],
        groups=combined.iloc[: len(train)]["batter_id"]
    ).fit(reml=False)

    # predict on test set
    pred = mdl.predict(exog=combined.iloc[len(train):])
    true = test["exit_velo"].values
    rmse = np.sqrt(np.mean((pred - true) ** 2))
    return mdl, rmse

if __name__ == "__main__":
    from src.data.load_data import load_and_clean_data
    from src.features.feature_engineering import feature_engineer
    from src.features.preprocess import prepare_for_mixed_and_hierarchical
    from sklearn.model_selection import train_test_split

    raw_path = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    df = load_and_clean_data(raw_path)
    df_fe = feature_engineer(df)

    # Prepare and split
    df_model = prepare_for_mixed_and_hierarchical(df_fe)
    train_df, test_df = train_test_split(df_model, test_size=0.2, random_state=42)
 
    # Fit mixed-effects
    mixed_model, rmse_mixed = fit_mixed(train_df, test_df)
    print(f"Mixed-effects model RMSE: {rmse_mixed:.4f}")

    # In the smoke test section: P-Value Checks for Mixed-Effects Models
    print("Mixed-effects model summary:\n", mixed_model.summary())

Overwriting src/models/mixed.py


In [18]:
%%writefile src/utils/bayesian_explainability.py
import arviz as az
import shap, numpy as np
import matplotlib.pyplot as plt

# ---------------- Posterior summaries -----------------
def plot_parameter_forest(idata, var_names=None, hdi_prob=0.95):
    """Caterpillar/forest plot of posterior estimates."""
    return az.plot_forest(
        idata,
        var_names=var_names,
        combined=True,
        hdi_prob=hdi_prob,
        kind="forest",
        figsize=(6, len(var_names or idata.posterior.data_vars) * 0.4),
    )

def posterior_table(idata, round_to=2):
    """
    Return a nicely rounded HDI/mean table with significance.
    """
    summary = az.summary(idata, hdi_prob=0.95).round(round_to)
    summary["significant"] = (summary["hdi_2.5%"] > 0) | (summary["hdi_97.5%"] < 0)
    return summary

# ---------------- Posterior‑predictive checks ---------
def plot_ppc(idata, kind="overlay"):
    """Visual PPC (over‑laid densities by default)."""
    return az.plot_ppc(idata, kind=kind, alpha=0.1)

# ---------------- SHAP-based feature importances ------
def shap_explain(predict_fn, background_df, sample_df):
    """
    Model‑agnostic Kernel SHAP on the *posterior mean predictor*.

    predict_fn(df) must return a 1‑D numpy array of predictions.
    """
    explainer = shap.KernelExplainer(predict_fn, background_df)
    shap_values = explainer.shap_values(sample_df, nsamples=200)
    shap.summary_plot(shap_values, sample_df, show=False)
    plt.tight_layout()
    return shap_values




Overwriting src/utils/bayesian_explainability.py


In [19]:
%%writefile src/utils/bayesian_metrics.py
import arviz as az
from sklearn.metrics import mean_squared_error,root_mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def compute_classical_metrics(idata, y_true):
    """Compute MSE, RMSE, MAE & R² from the posterior predictive distribution."""
    # Extract posterior predictive draws and compute mean prediction
    y_ppc = (
        idata.posterior_predictive['y_obs']
        .stack(samples=('chain', 'draw'))
        .values
    )
    y_pred = y_ppc.mean(axis=1)

    # Classical regression metrics
    mse = mean_squared_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    # Print results
    print(f"▶ Classical MSE : {mse:.2f}")
    print(f"▶ Classical RMSE: {rmse:.2f}")
    print(f"▶ Classical MAE : {mae:.2f}")
    print(f"▶ Classical R²  : {r2:.3f}")

    return {'mse': mse, 'rmse': rmse, 'mae': mae, 'r2': r2}



def compute_bayesian_metrics(idata):
    """Compute PSIS-LOO, WAIC and Pareto k diagnostics for model comparison."""
    # 1) Guard missing log_likelihood
    if 'log_likelihood' not in idata.groups():
        print("⚠️ InferenceData has no log_likelihood; skipping LOO/WAIC.")
        return None

    try:
        loo  = az.loo(idata, pointwise=True)
        waic = az.waic(idata, pointwise=True)
    except Exception as e:
        print(f"⚠️ LOO/WAIC computation failed: {e}")
        return None

    # 2) Extract scalars by indexing the ELPDData (a pandas Series)
    #    LOOIC = -2 * elpd_loo
    looic    = -2 * loo['elpd_loo']
    p_loo    = loo['p_loo']

    #    WAIC  = -2 * elpd_waic
    waic_val = -2 * waic['elpd_waic']
    p_waic   = waic['p_waic']

    #    Pareto k diagnostic: proportion of observations with k > 0.7
    prop_bad_k = np.mean(loo['pareto_k'].values > 0.7)

    # 3) Print results
    print(f"▶ LOOIC       : {looic:.1f}, p_loo: {p_loo:.1f}")
    print(f"▶ WAIC        : {waic_val:.1f}, p_waic: {p_waic:.1f}")
    print(f"▶ Pareto k>0.7: {prop_bad_k:.2%} of observations")

    return {'loo': loo, 'waic': waic, 'prop_bad_k': prop_bad_k}




def compute_convergence_diagnostics(idata):
    """Print R̂ and ESS bulk/tail for key parameters."""
    if 'posterior' not in idata.groups():
        print("No posterior group in InferenceData; skipping convergence diagnostics.")
        return None

    try:
        summary = az.summary(
            idata,
            var_names=["mu", "beta", "sigma_e"],
            kind="diagnostics",
            round_to=2
        )
    except KeyError as e:
        print(f"Convergence diagnostics skipped: missing vars {e}")
        return None

    print("▶ Convergence diagnostics (R̂, ESS):")
    print(summary[["r_hat", "ess_bulk", "ess_tail"]])
    return summary


def compute_calibration(idata, y_true, hdi_prob=0.95):
    """
    Compute calibration: the fraction of true observations that lie within the posterior predictive HDI.
    """
    # Extract posterior predictive draws
    y_ppc = (
        idata.posterior_predictive['y_obs']
        .stack(samples=('chain','draw'))
        .values
    )
    # Compute lower/upper quantiles per observation
    lower = np.percentile(y_ppc, (1-hdi_prob)/2*100, axis=1)
    upper = np.percentile(y_ppc, (1+(hdi_prob))/2*100, axis=1)
    within = ((y_true >= lower) & (y_true <= upper)).mean()
    print(f"▶ Calibration: {within:.2%} of true values within {int(hdi_prob*100)}% HDI")
    return within

if __name__ == "__main__":
    import numpy as np
    import arviz as az

    # 1) True values
    np.random.seed(42)
    y_true = np.random.normal(0, 1, size=10)

    # 2) Posterior predictive draws (2 chains × 5 draws × 10 obs)
    y_obs = np.random.normal(loc=y_true, scale=0.5, size=(2,5,10))

    # 3) Posterior parameters μ, β, σₑ (2 chains × 5 draws)
    posterior = {
        'mu':      np.random.normal(0, 1, size=(2,5)),
        'beta':    np.random.normal(0, 1, size=(2,5)),
        'sigma_e': np.abs(np.random.normal(1, 0.2, size=(2,5))),
    }

    # 4) Log‑likelihood (for each chain, draw, obs)
    log_lik = np.random.normal(-1, 0.5, size=(2,5,10))

    # 5) Build InferenceData
    idata = az.from_dict(
        posterior=posterior,
        posterior_predictive={'y_obs': y_obs},
        log_likelihood     ={'y_obs': log_lik},
        dims={
            'y_obs': ['chain','draw','obs'],
            'mu':    ['chain','draw'],
            'beta':  ['chain','draw'],
            'sigma_e':['chain','draw'],
        },
        coords={
            'chain': [0,1],
            'draw':  list(range(5)),
            'obs':   list(range(10))
        }
    )

    print("=== Classical Metrics ===")
    compute_classical_metrics(idata, y_true)

    print("\n=== Calibration ===")
    compute_calibration(idata, y_true)

    print("\n=== Bayesian Metrics ===")
    compute_bayesian_metrics(idata)

    print("\n=== Convergence Diagnostics ===")
    compute_convergence_diagnostics(idata)


Overwriting src/utils/bayesian_metrics.py


In [20]:
%%writefile src/utils/hierarchical_utils.py

import arviz as az
import joblib
from pathlib import Path

def save_model(idata, file_path: str, overwrite: bool = True):
    """Save ArviZ InferenceData to NetCDF."""
    idata.to_netcdf(file_path, engine="h5netcdf", overwrite_existing=overwrite)
    print(f"✔︎ saved model → {file_path}")

def load_model(file_path: str):
    """Load ArviZ InferenceData from NetCDF."""
    idata = az.from_netcdf(file_path, engine="h5netcdf")
    print(f"✔︎ loaded model ← {file_path}")
    return idata

def save_preprocessor(transformer, file_path: str):
    """Save the scikit-learn transformer to a joblib file."""
    joblib.dump(transformer, file_path)
    print(f"✔︎ saved preprocessor → {file_path}")

def load_preprocessor(file_path: str):
    """Load the scikit-learn transformer from a joblib file."""
    transformer = joblib.load(file_path)
    print(f"✔︎ loaded preprocessor ← {file_path}")
    return transformer

if __name__ == "__main__":
    # === Editable settings ===
    # Path to the saved model (NetCDF format)
    MODEL_PATH = "data/models/saved_models/model.nc"
    # Path to the preprocessor
    PREPROC_PATH = "data/models/saved_models/preprocessor.joblib"
    # Input data for prediction (raw CSV with exit velocity data)
    raw_path = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    # Output predictions file (CSV) or set to None to print to console
    OUTPUT_PREDS_2024 = "data/predictions/predictions_2024.csv"  # <-- EDITABLE: set output CSV path or None
    
    model = load_model(MODEL_PATH)
    save_model(model, MODEL_PATH)



Overwriting src/utils/hierarchical_utils.py


In [21]:
%%writefile src/utils/posterior.py
# src/utils/posterior.py
import numpy as np
import pandas as pd
import arviz as az

# ── REPLACEMENT: paste over the whole function ─────────────────────────
import json, pathlib, numpy as np, pandas as pd, arviz as az

JSON_GLOBAL = pathlib.Path("data/models/saved_models/global_effects.json")

def posterior_to_frame(idata: az.InferenceData) -> pd.DataFrame:
    """
    Returns a batter‑level summary **AND** writes global effects to JSON.

    File written  ➜  data/models/saved_models/global_effects.json
    """
    post = idata.posterior

    # ------- per‑batter u summaries -----------------------------------
    u   = post["u"]                                         # (chain,draw,batter)
    stats = {
        "u_mean"  : u.mean(("chain","draw")).values,
        "u_sd"    : u.std (("chain","draw")).values,
        "u_q2.5"  : np.percentile(u.values,  2.5, axis=(0,1)),
        "u_q50"   : np.percentile(u.values, 50.0, axis=(0,1)),
        "u_q97.5" : np.percentile(u.values,97.5, axis=(0,1)),
    }
    df = pd.DataFrame({"batter_idx": np.arange(u.shape[-1]), **stats})

    # ------- global effects ------------------------------------------
    mu_mean = post["mu"].mean().item()

    # β_age  ➜ last entry of beta vector (age_centered was added last)
    beta  = post["beta"]
    feat_dim = [d for d in beta.dims if d not in ("chain","draw")][0]
    beta_age = beta.isel({feat_dim: -1}).mean().item()

    beta_level = post["beta_level"].mean(("chain","draw")).values.tolist()
    sigma_b    = post["sigma_b"].mean().item()
    sigma_e    = post["sigma_e"].mean().item()

    global_eff = dict(
        mu_mean=mu_mean,
        beta_age=beta_age,
        beta_level=beta_level,
        sigma_b=sigma_b,
        sigma_e=sigma_e,
        median_age=idata.attrs.get("median_age", 26.0),
        beta=post["beta"].mean(("chain","draw")).values.tolist(),  # Save all beta coefficients
        feature_names=idata.attrs.get("feature_names", [])  # Also save feature names if available
    )

    # ➜  write side‑car JSON (overwrite every run)
    JSON_GLOBAL.write_text(json.dumps(global_eff, indent=2))
    print(f"✔︎ wrote global effects → {JSON_GLOBAL}")

    return df





def align_batter_codes(df_roster: pd.DataFrame,
                       train_cats: pd.Index) -> pd.Series:
    """
    Convert integer batter_ids in *roster* into the categorical codes
    **identical** to what the model saw during training.

    Any unseen batter gets code = -1 (handled later).
    """
    cat = pd.Categorical(df_roster["batter_id"], categories=train_cats)
    return pd.Series(cat.codes, index=df_roster.index)



Overwriting src/utils/posterior.py


In [22]:
%%writefile src/utils/validation.py
"""
Generic K‑fold validator.

• Works for sklearn Pipelines *or* PyMC idata.
• Decides how to extract predictions based on
  the object returned by `fit_func`.
"""
from __future__ import annotations
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from arviz import InferenceData
import arviz as az
from typing import Callable, List, Union
import statsmodels as sm



def _rmse(a: np.ndarray, b: np.ndarray) -> float:
    return np.sqrt(np.mean((a - b) ** 2))


def run_kfold_cv(
    fit_func: Callable[[pd.DataFrame, pd.DataFrame], tuple],
    df: pd.DataFrame,
    k: int = 5,
    random_state: int = 0,
    **fit_kw
) -> List[float]:
    """
    Sklearn-style K-fold CV:
      fit_func(train_df, test_df, **fit_kw) -> (model, rmse)
    Returns a list of rmse scores.
    """
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
    rmses: List[float] = []
    for train_idx, test_idx in kf.split(df):
        train, test = df.iloc[train_idx], df.iloc[test_idx]
        _, rmse = fit_func(train, test, **fit_kw)
        rmses.append(rmse)
    return rmses



def run_kfold_cv_pymc(
    fit_func: Callable[[pd.DataFrame], InferenceData],
    df: pd.DataFrame,
    k: int = 5,
    random_state: int = 0
) -> list[float]:
    """
    PyMC K-fold CV: 
      fit_func(train_df) -> ArviZ InferenceData with posterior a[group] & sigma.
    Returns RMSE on each hold-out fold.
    """
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
    rmses: list[float] = []

    for train_idx, test_idx in kf.split(df):
        train_df, test_df = df.iloc[train_idx], df.iloc[test_idx]
        idata = fit_func(train_df)

        # 1) grab posterior samples of the group intercepts and noise σ
        #    stack chain+draw into one dim of length M
        a_samples = (
            idata.posterior["a"]
            .stack(sample=("chain", "draw"))    # dims now ("group","sample")
            .transpose("sample", "group")       # shape (M, n_groups)
            .values
        )
        sigma_samples = (
            idata.posterior["sigma"]
            .stack(sample=("chain", "draw"))    # shape (M,)
            .values
        )

        # 2) for each test row, pull the intercept for its group
        groups = test_df["group"].values       # shape (n_test,)
        # a_samples[:, groups] → shape (M, n_test)
        pred_samples = a_samples[:, groups]

        # 3) point estimate = posterior mean for each test row
        pred_mean = pred_samples.mean(axis=0)  # shape (n_test,)

        # 4) compute RMSE against true exit_velo
        true_vals = test_df["exit_velo"].values
        rmse = np.sqrt(np.mean((pred_mean - true_vals) ** 2))
        rmses.append(rmse)

    return rmses




# helper to score a *single* train/test split for idata
def rmse_pymc(idata: InferenceData, test_df: pd.DataFrame) -> float:
    """
    Compute RMSE by comparing posterior-mean predictions
    (using the group intercept `a`) against observed `exit_velo`.
    """
    # 1) Stack (chain, draw) into a single "sample" dimension
    a_samples = (
        idata.posterior["a"]
        .stack(sample=("chain", "draw"))    # dims → ("group","sample")
        .transpose("sample", "group")       # shape → (M, n_groups)
        .values
    )

    # 2) Gather the sampled intercepts for each test row
    groups = test_df["group"].values       # shape → (n_test,)
    pred_samples = a_samples[:, groups]    # shape → (M, n_test)

    # 3) Posterior mean prediction for each test instance
    pred_mean = pred_samples.mean(axis=0)  # shape → (n_test,)

    # 4) Compute and return RMSE versus the true values
    true_vals = test_df["exit_velo"].values
    return np.sqrt(np.mean((pred_mean - true_vals) ** 2))


def posterior_predictive_check(idata, df, batter_idx):
    """
    Plot observed vs. simulated exit-velo histograms.
    """
    import matplotlib.pyplot as plt
    obs = df["exit_velo"].values
    sim = (
        idata.posterior_predictive["y_obs"]
        .stack(samples=("chain", "draw"))
        .values.flatten()
    )

    fig, ax = plt.subplots(1, 2, figsize=(8, 3))
    ax[0].hist(obs, bins=30)
    ax[0].set_title("Observed")
    ax[1].hist(sim, bins=30)
    ax[1].set_title("Simulated")
    fig.tight_layout()
    return fig




def prediction_interval(model, X, alpha=0.05, method='linear'):
    """
    Compute prediction intervals for a model.
    """
    if method == 'linear':
        # For OLS and Ridge
        X_const = sm.add_constant(X)
        preds = model.get_prediction(X_const)
        pred_int = preds.conf_int(alpha=alpha)
        return preds.predicted_mean, pred_int[:, 0], pred_int[:, 1]
    elif method == 'bayesian':
        # For Bayesian models
        hdi = az.hdi(model, hdi=1 - alpha)
        return (
            hdi.posterior_predictive.y_obs.sel(hdi=f"{alpha/2*100}%"),
            hdi.posterior_predictive.y_obs.sel(hdi=f"{(1-alpha/2)*100}%")
        )
    elif method == 'gbm':
        # For XGBoost quantile regression
        lower = model.predict(X, pred_contribs=False, iteration_range=(0, model.best_iteration))
        upper = model.predict(X, pred_contribs=False, iteration_range=(0, model.best_iteration))
        return lower, upper  # Replace with actual quantile regression
    else:
        raise ValueError("Method not supported")

# Example for bootstrapping GBM
def bootstrap_prediction_interval(model, X, n_bootstraps=1000, alpha=0.05):
    """
    Compute nonparametric bootstrap confidence intervals for predictions.

    Must support either:
      - model.predict(X) -> array_like shape (n_obs,)
      - model.get_prediction(X).predicted_mean -> array_like shape (n_obs,)

    Returns lower, upper arrays of shape (n_obs,) at the given alpha level.
    """
    import numpy as np

    # Determine number of observations
    try:
        n_obs = X.shape[0]
    except Exception:
        raise ValueError("X must have a valid `.shape[0]`")

    # Allocate storage
    all_preds = np.zeros((n_bootstraps, n_obs))

    for i in range(n_bootstraps):
        # Sample indices with replacement
        idx = np.random.randint(0, n_obs, size=n_obs)
        sampled = X.iloc[idx] if hasattr(X, "iloc") else X[idx]

        # Dispatch to correct predict method
        if hasattr(model, "predict"):
            preds_i = model.predict(sampled)
        elif hasattr(model, "get_prediction"):
            result = model.get_prediction(sampled)
            preds_i = getattr(result, "predicted_mean", None)
            if preds_i is None:
                raise AttributeError(
                    "get_prediction(...) did not return `predicted_mean`"
                )
        else:
            raise AttributeError(
                f"Model {model!r} has no `predict` or `get_prediction` method"
            )

        all_preds[i, :] = preds_i

    # Compute CI percentiles
    lower = np.percentile(all_preds, 100 * (alpha / 2), axis=0)
    upper = np.percentile(all_preds, 100 * (1 - alpha / 2), axis=0)
    return lower, upper






if __name__ == "__main__":
    import numpy as np
    import pandas as pd
    import arviz as az
    import statsmodels.api as sm
    import pymc as pm
    from sklearn.linear_model import LinearRegression

    # from src.utils.validation import (
    # run_kfold_cv_pymc, prediction_interval, 
    # bootstrap_prediction_interval, rmse_pymc, 
    # posterior_predictive_check
    # )

    # 1) Synthetic data for sklearn
    np.random.seed(42)
    X = np.linspace(0, 10, 40)
    y = 2 * X + 1 + np.random.normal(0, 1, size=X.shape)
    df_skl = pd.DataFrame({"feature": X, "exit_velo": y})

    def fit_sklearn(train, test):
        X_tr, y_tr = train[["feature"]], train["exit_velo"]
        X_te, y_te = test[["feature"]], test["exit_velo"]
        m = LinearRegression().fit(X_tr, y_tr)
        preds = m.predict(X_te)
        rmse = np.sqrt(np.mean((preds - y_te) ** 2))
        return m, rmse

    print("\n--- sklearn CV ---")
    print("RMSEs:", run_kfold_cv(fit_sklearn, df_skl, k=4))

    # 2) Synthetic hierarchical data for PyMC
    #    (3 groups, group‐level intercepts)
    n_per = 20
    groups = np.repeat(np.arange(3), n_per)
    true_alpha = np.array([1.0, 3.0, 5.0])
    y_hier = true_alpha[groups] + np.random.normal(0, 1, size=groups.size)
    df_hier = pd.DataFrame({"exit_velo": y_hier, "group": groups})

    def fit_hierarchical(train_df: pd.DataFrame) -> az.InferenceData:
        """
        Fit a hierarchical model and append posterior_predictive samples for y_obs.
        Returns:
            InferenceData with .posterior and .posterior_predictive["y_obs"].
        """
        coords = {"group": np.unique(train_df["group"])}
        with pm.Model(coords=coords) as model:
            mu_a = pm.Normal("mu_a", 0, 5)
            sigma_a = pm.HalfNormal("sigma_a", 5)
            a = pm.Normal("a", mu=mu_a, sigma=sigma_a, dims="group")
            sigma = pm.HalfNormal("sigma", 1)
            y_grp = a[train_df["group"].values]
            pm.Normal("y_obs", mu=y_grp, sigma=sigma,
                    observed=train_df["exit_velo"].values)

            # 1) Draw posterior samples
            idata = pm.sample(
                draws=500, tune=500,
                chains=2, cores=1,
                return_inferencedata=True,
                progressbar=False
            )

            # 2) Generate posterior_predictive samples for 'y_obs'
            pm.sample_posterior_predictive(
                idata,
                model=model,
                var_names=["y_obs"],
                extend_inferencedata=True,
                random_seed=42,
                progressbar=False
            )

        # 3) Return the enriched InferenceData
        return idata


    print("\n--- PyMC hierarchical CV ---")
    bayes_rmse = run_kfold_cv_pymc(fit_hierarchical, df_hier, k=3)
    print("Bayesian RMSEs:", bayes_rmse)

    # 3) Test prediction_interval (linear OLS)
    df_ols = df_skl.copy()
    df_ols["const"] = 1.0
    ols = sm.OLS(df_ols["exit_velo"], df_ols[["const", "feature"]]).fit()
    mean, lo, hi = prediction_interval(ols, df_skl[["feature"]], method="linear")
    print("\n--- OLS prediction_interval shapes ---")
    print("mean:", mean.shape, "lower:", lo.shape, "upper:", hi.shape)

    # 4) Test bootstrap_prediction_interval with LinearRegression
    lr = LinearRegression().fit(df_skl[["feature"]], df_skl["exit_velo"])
    lower, upper = bootstrap_prediction_interval(lr, df_skl[["feature"]], n_bootstraps=200)
    print("\n--- bootstrap_prediction_interval shapes ---")
    print("lower:", lower.shape, "upper:", upper.shape)

    # 5) rmse_pymc + posterior_predictive_check on a single InferenceData
    #    (we can re-use the last fold of the hierarchical example)
    last_idata = fit_hierarchical(df_hier)
    rmse_val = rmse_pymc(last_idata, df_hier)
    print("\n--- rmse_pymc on full data ---", rmse_val)

    # 6) posterior_predictive_check plot
    print("\n--- posterior_predictive_check (figure) ---")
    fig = posterior_predictive_check(last_idata, df_hier, batter_idx=None)
    fig.savefig("ppc_hist.png")
    print("Saved histogram to ppc_hist.png")


Overwriting src/utils/validation.py


In [23]:
%%writefile src/utils/jax_gpu_utils.py

import os, subprocess, json, logging, platform, jax
from jax.lib import xla_bridge

def gpu_diagnostics():
    info = {
        "backend":      xla_bridge.get_backend().platform,
        "devices":      [str(d) for d in jax.devices()],
        "python":       platform.python_version(),
        "ld_library_path": os.getenv("LD_LIBRARY_PATH","<unset>"),
    }
    if shutil.which("nvidia-smi"):
        try:
            smi = subprocess.check_output(
                ["nvidia-smi", "--query-gpu=name,driver_version,memory.total",
                 "--format=csv,noheader,nounits"]
            )
            info["nvidia-smi"] = smi.decode().strip()
        except Exception as exc:
            info["nvidia-smi-error"] = repr(exc)
    return info

def log_gpu_diagnostics(level=logging.INFO):
    logging.getLogger(__name__).log(
        level,
        "GPU-diag: %s",
        json.dumps(gpu_diagnostics(), indent=2)
    )


Overwriting src/utils/jax_gpu_utils.py


In [24]:
%%writefile src/models/bayesian_alternatives.py

# ─────────────────────────────────────────────────────────────
# src/models/bayesian_alternatives.py
# Implements additional Bayesian engines that all return
# arviz.InferenceData so downstream metrics remain unchanged.
# ─────────────────────────────────────────────────────────────
from __future__ import annotations
from pathlib import Path
from typing import Tuple
import tempfile
import os
from cmdstanpy import CmdStanModel  # Stan program wrapper
import numpy as np
import pandas as pd
import arviz as az
import pyjags   
import tensorflow as tf
import tensorflow_probability as tfp  
# 1)  CmdStanPy  ---------------------------------------------------------
def fit_bayesian_cmdstanpy(
    stan_code: str,
    stan_data: dict,
    *,
    draws: int = 1000,
    warmup: int = 500,
    chains: int = 4,
    seed: int = 42,
) -> az.InferenceData:
    """
    Compile a Stan model from source code string and return ArviZ InferenceData.
    This writes the code to a temporary file, compiles it, samples, and cleans up.
    """
    # 1) Write the Stan code to a temp file
    with tempfile.NamedTemporaryFile(
        mode="w", suffix=".stan", delete=False
    ) as tmp:
        tmp.write(stan_code)
        stan_file = tmp.name  # Path to pass to CmdStanModel

    try:
        # 2) Instantiate and compile the model from the .stan file
        model = CmdStanModel(
            stan_file=stan_file,
            force_compile=True
        )

        # 3) Draw samples using HMC/NUTS
        fit = model.sample(
            data=stan_data,
            iter_sampling=draws,
            iter_warmup=warmup,
            chains=chains,
            seed=seed
        )

        # 4) Convert to ArviZ InferenceData
        idata = az.from_cmdstanpy(posterior=fit)
        return idata

    finally:
        # 5) Cleanup temp files to prevent clutter
        try:
            os.remove(stan_file)
        except OSError:
            pass



# 2)  PyJAGS (Gibbs)  ----------------------------------------------------
def fit_bayesian_pyjags(
    jags_model: str,
    jags_data: dict,
    *,
    draws: int = 5000,
    burn: int = 1000,
    thin: int = 1,
    chains: int = 4,
    seed: int = 42,
) -> az.InferenceData:
    """
    Fit a JAGS model via PyJAGS and convert to InferenceData.
    """                                               # PyJAGS interface :contentReference[oaicite:10]{index=10}:contentReference[oaicite:11]{index=11}
    model = pyjags.Model(code=jags_model,
                         data=jags_data,
                         chains=chains,
                         adapt=burn,
                         init=[{".RNG.seed": seed + c * 10} for c in range(chains)])
    samples = model.sample(draws, vars=None, thin=thin)
    idata   = az.from_pyjags(posterior=samples)
    return idata


# 3)  NumPyro (NUTS)  ----------------------------------------------------
def fit_bayesian_numpyro(
    numpyro_model,
    rng_key,
    *,
    draws: int = 1000,
    warmup: int = 500,
    chains: int = 4,
) -> az.InferenceData:
    """
    Run NUTS in NumPyro; return InferenceData (auto-vectorises across chains).
    """
    import jax
    from numpyro.infer import MCMC, NUTS                           # NumPyro HMC/NUTS :contentReference[oaicite:12]{index=12}:contentReference[oaicite:13]{index=13}
    kernel = NUTS(numpyro_model)
    mcmc   = MCMC(kernel,
                  num_warmup=warmup,
                  num_samples=draws,
                  num_chains=chains,
                  progress_bar=False)
    mcmc.run(rng_key)
    idata  = az.from_numpyro(mcmc)
    return idata


# 4)  TensorFlow Probability HMC  ---------------------------------------
def fit_bayesian_tfp_hmc(
    target_log_prob_fn,
    init_state,
    *,
    step_size: float = 0.05,
    leapfrog_steps: int = 5,
    draws: int = 1000,
    burnin: int = 500,
    seed: int = 42,
) -> az.InferenceData:
    """
    Vanilla single-chain HMC in TensorFlow Probability.
    """                         # TFP HMC kernel :contentReference[oaicite:14]{index=14}:contentReference[oaicite:15]{index=15}
    tfd, tfmcmc = tfp.distributions, tfp.mcmc

    hmc = tfmcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=target_log_prob_fn,
        step_size=step_size,
        num_leapfrog_steps=leapfrog_steps)

    adaptive_hmc = tfmcmc.SimpleStepSizeAdaptation(
        inner_kernel=hmc,
        num_adaptation_steps=int(0.8 * burnin))

    @tf.function(autograph=False, jit_compile=True)
    def _run_chain():
        return tfmcmc.sample_chain(
            num_results=draws,
            current_state=init_state,
            kernel=adaptive_hmc,
            num_burnin_steps=burnin,
            seed=seed)

    samples, _ = _run_chain()
    # Wrap in dict so ArviZ can coerce
    posterior = {f"param_{i}": s.numpy() for i, s in enumerate(samples)}
    idata     = az.from_dict(posterior=posterior)
    return idata


# 5)  PyMC ADVI (fast VI baseline)  --------------------------------------
def fit_bayesian_pymc_advi(
    pymc_model,
    *,
    draws: int = 1000,
    tune: int = 10000,
) -> az.InferenceData:
    """
    Run Automatic Differentiation VI (ADVI) in PyMC —> InferenceData.
    """
    import pymc as pm                                              # PyMC ADVI API :contentReference[oaicite:16]{index=16}:contentReference[oaicite:17]{index=17}
    with pymc_model:
        approx = pm.fit(n=tune, method="advi", progressbar=False)  # returns Approximation
        idata  = approx.sample(draws)
    return idata

# ───────────────────────────────────────────────────────────────────────
# 6. Smoke test (only run when module executed directly)
# ----------------------------------------------------------------------
if __name__ == "__main__":
    import jax.random as jr
    import tensorflow_probability as tfp
    import tensorflow as tf
    import pymc as pm
    from cmdstanpy import CmdStanModel
    import arviz as az
    import matplotlib.pyplot as plt
    from pathlib import Path
    import numpy as np

    # ─── User-configurable sampling parameters ────────────────────────
    N_CHAINS   = 4      # number of MCMC chains (PyMC, Stan, NumPyro, TFP)
    N_DRAWS    = 500   # number of samples to keep per chain
    N_TUNE     = 500   # number of tuning/adaptation steps per chain
    PP_DRAWS   = 250    # draws for posterior_predictive in PyMC-ADVI & attach_ppc
    JAGS_DRAWS = 500   # total samples for PyJAGS
    JAGS_BURN  = 500   # adaptation steps for PyJAGS
    SEED       = 42     # global RNG seed
    # ────────────────────────────────────────────────────────────────────

    # ─── Synthetic data ────────────────────────────────────────────────
    N       = 50
    rng     = np.random.default_rng(SEED)
    x_data  = rng.uniform(-2, 2, size=N)
    y_true  = 1.0 + 2.5 * x_data
    y_obs   = y_true + rng.normal(0, 0.8, size=N)

    # ─── Helper: ensure posterior_predictive["y_obs"] exists ───────────
    def _attach_ppc(idata: az.InferenceData,
                    x: np.ndarray,
                    alpha: str = "alpha",
                    beta: str = "beta",
                    sigma: str = "sigma",
                    draws: int | None = None) -> None:
        """
        Unified, robust method to add posterior predictive samples to idata.
        Ensures a uniform (chain, draw, obs) array is passed to ArviZ.
        """
        post = idata.posterior

        # 1) Extract posterior arrays via stacking
        a = post[alpha].stack(samples=("chain", "draw")).values  # shape: (n_chains*n_draws,)
        b = post[beta].stack(samples=("chain", "draw")).values
        s = post[sigma].stack(samples=("chain", "draw")).values

        # 2) Determine dimensions
        n_chains = post.sizes.get("chain", 1)                  # single‐chain ADVI fallback
        n_draws  = post.sizes["draw"]
        n_obs    = x.shape[0]
        M        = min(a.size, draws) if draws is not None else a.size

        # 3) Build predictive draws (shape: M × n_obs)
        mu  = a[:M, None] + b[:M, None] * x[None, :]
        ypp = rng.normal(loc=mu, scale=s[:M, None])

        # 4) Reshape to (chain, draw, obs)
        arr = ypp.reshape(n_chains, n_draws, n_obs)

        # 5) Attach to InferenceData, letting ArviZ handle chain/draw dims
        idata.add_groups(
            posterior_predictive={"y_obs": arr},
            dims={"y_obs": ["obs"]},
        )



    # ─── 1) PyMC HMC (NUTS) ───────────────────────────────────────────
    coords = {"obs": np.arange(N)}
    with pm.Model(coords=coords) as pymc_model:
        alpha = pm.Normal("alpha", 0, 5)
        beta  = pm.Normal("beta",  0, 5)
        sigma = pm.HalfNormal("sigma", 1)
        mu    = alpha + beta * x_data
        pm.Normal("y", mu, sigma, observed=y_obs, dims="obs")

    idata_pymc_hmc = pm.sample(
        N_DRAWS,
        tune=N_TUNE,
        chains=N_CHAINS,
        random_seed=SEED,
        progressbar=False,
        return_inferencedata=True,
        model=pymc_model
    )
    pm.sample_posterior_predictive(
        idata_pymc_hmc,
        var_names=["y"],
        extend_inferencedata=True,
        model=pymc_model
    )
    idata_pymc_hmc.posterior_predictive = \
        idata_pymc_hmc.posterior_predictive.rename({"y": "y_obs"})

    # ─── 2) PyMC ADVI ─────────────────────────────────────────────────
    idata_pymc_advi = fit_bayesian_pymc_advi(pymc_model,
                                             draws=PP_DRAWS,
                                             tune=N_TUNE*5)
    _attach_ppc(idata_pymc_advi, x_data, draws=PP_DRAWS)

    # ─── 3) CmdStanPy (Stan HMC) ──────────────────────────────────────
    stan_code = """
    data { int<lower=0> N; vector[N] x; vector[N] y; }
    parameters { real alpha; real beta; real<lower=0> sigma; }
    model { y ~ normal(alpha+beta*x, sigma); }
    generated quantities {
      vector[N] y_obs;
      for (n in 1:N)
        y_obs[n] = normal_rng(alpha+beta*x[n], sigma);
    }
    """
    stan_data = {"N": N, "x": x_data, "y": y_obs}
    idata_stan = fit_bayesian_cmdstanpy(stan_code,
                                        stan_data,
                                        draws=N_DRAWS,
                                        warmup=N_TUNE,
                                        chains=N_CHAINS,
                                        seed=SEED)

    # ─── 4) PyJAGS (Gibbs Sampling) ───────────────────────────────────
    jags_model = """
    model {
      for (i in 1:N) {
        y[i] ~ dnorm(alpha+beta*x[i], tau);
        y_obs[i] <- alpha + beta*x[i] + sigma*randn();
      }
      alpha ~ dnorm(0, .001);
      beta  ~ dnorm(0, .001);
      sigma ~ dunif(0, 10);
      tau   <- pow(sigma, -2);
    }
    """
    jags_data = {"N": N, "x": x_data, "y": y_obs}
    idata_jags = fit_bayesian_pyjags(jags_model,
                                     jags_data,
                                     draws=JAGS_DRAWS,
                                     burn=JAGS_BURN,
                                     chains=N_CHAINS,
                                     seed=SEED)
    _attach_ppc(idata_jags, x_data, draws=PP_DRAWS)

    # ─── 5) NumPyro (NUTS) ─────────────────────────────────────────────
    import numpyro, numpyro.distributions as dist
    def numpyro_model(x, y=None):
        a = numpyro.sample("alpha", dist.Normal(0,5))
        b = numpyro.sample("beta",  dist.Normal(0,5))
        s = numpyro.sample("sigma", dist.HalfNormal(1))
        mu = a + b * x
        numpyro.sample("y", dist.Normal(mu, s), obs=y)
    rng_key = jr.PRNGKey(SEED)
    idata_numpyro = fit_bayesian_numpyro(numpyro_model,
                                         rng_key,
                                         draws=N_DRAWS,
                                         warmup=N_TUNE,
                                         chains=N_CHAINS)
    _attach_ppc(idata_numpyro, x_data, draws=PP_DRAWS)

    # ─── 6) TFP HMC ───────────────────────────────────────────────────
    tfd, tfmcmc = tfp.distributions, tfp.mcmc
    def target_log_prob_fn(alpha, beta, log_sigma):
        sigma = tf.exp(log_sigma)
        yhat  = alpha + beta * tf.constant(x_data, tf.float32)
        return tf.reduce_sum(tfd.Normal(yhat, sigma).log_prob(y_obs)) + \
               tfd.Normal(0,5).log_prob(alpha) + \
               tfd.Normal(0,5).log_prob(beta) + \
               tfd.Normal(0,1).log_prob(log_sigma)

    init_state = [tf.zeros([]), tf.zeros([]), tf.zeros([])]
    idata_tfp  = fit_bayesian_tfp_hmc(target_log_prob_fn,
                                      init_state,
                                      draws=N_DRAWS,
                                      burnin=N_TUNE,
                                      seed=SEED)
    _attach_ppc(idata_tfp, x_data, sigma="param_2", draws=PP_DRAWS)

    # ─── Compare & Plot ───────────────────────────────────────────────
    from src.utils.bayesian_metrics import compute_classical_metrics
    engines = {
      "PyMC-HMC":  idata_pymc_hmc,
      "PyMC-ADVI": idata_pymc_advi,
      "Stan":      idata_stan,
      "JAGS":      idata_jags,
      "NumPyro":   idata_numpyro,
      "TFP-HMC":   idata_tfp,
    }
    for name,idata in engines.items():
        print(f"\n▼▼ {name}")
        compute_classical_metrics(idata, y_obs)

    # quick visual check (posterior mean lines)
    plt.figure(figsize=(6,4))
    plt.scatter(x_data, y_obs, label="obs", alpha=.6)
    line_x = np.linspace(-2, 2, 100)
    colors = plt.cm.tab10(np.linspace(0,1,len(engines)))
    for c,(name,id_) in zip(colors, engines.items()):
        a = id_.posterior["alpha"].mean().item()
        b = id_.posterior["beta"].mean().item()
        plt.plot(line_x, a + b*line_x, color=c, label=name, lw=1.2)
    plt.legend(); plt.title("Posterior mean fits – smoke test")
    plt.tight_layout()
    Path("smoke_test_fits.png").write_bytes(plt.savefig("/tmp/_smoke.png") or b"")
    print("\n✔︎ Smoke-test figure saved → smoke_test_fits.png")


Overwriting src/models/bayesian_alternatives.py


In [25]:
%%writefile src/models/hierarchical.py

import pymc as pm
import arviz as az
import numpy as np
import json  # Add json import for allocation result
from sklearn.compose import ColumnTransformer
from src.data.ColumnSchema import _ColumnSchema
from src.features.preprocess import transform_preprocessor

# ─── Apply our JAX GPU-memory flags first ───────────────────────────────
# this must run before any `import jax`
from src.utils.jax_memory_fix_module import apply_jax_memory_fix
# Configure JAX to use 90% of GPU memory with preallocation
apply_jax_memory_fix(fraction=0.9, preallocate=True)
# ─── Now it's safe to pull in JAX (with your flags applied) ─────────────
import jax, jaxlib

import logging
import pymc.sampling.jax as pmjax

import time
from contextlib import contextmanager
from tqdm.auto import tqdm
from jax.lib import xla_bridge
from src.utils.jax_gpu_utils import log_gpu_diagnostics

# Import memory monitoring utilities
from src.utils.jax_memory_monitor import (
    monitor_memory_usage,
    take_memory_snapshot,
    print_memory_snapshot,
    force_allocation_if_needed,
    generate_memory_report
)

# Log GPU diagnostics at startup
log_gpu_diagnostics()

# ── Context manager for timing ─────────────────────────────────────────
@contextmanager
def _timed_section(label: str):
    t0 = time.time()
    yield
    print(f"[{label}] finished in {time.time() - t0:,.1f} s")


# Attempt to import and configure JAX
USE_JAX = True
try:
    # Debug: confirm module and path
    print(f"🔍 JAX module: {jax!r}")
    print(f"🔍 JAX path:   {getattr(jax, '__file__', 'builtin')}")
    if not hasattr(jax, "__version__"):
        raise ImportError("jax.__version__ missing—possible circular import")
    print(f"✅ JAX version: {jax.__version__}")
    jax.config.update("jax_enable_x64", True)
    log_gpu_diagnostics() 
    # ── NEW: Verify PyMC sees GPU backend ─────────────────────────────
    print("🔍 PyMC version:", pm.__version__)
    print("🔍 JAX backend:", xla_bridge.get_backend().platform)

except Exception as e:
    USE_JAX = False
    print(f"⚠️  Warning: could not import/configure JAX ({e}). Falling back to CPU sampling.")


def fit_bayesian_hierarchical(
    df_raw,
    transformer: ColumnTransformer,
    batter_idx: np.ndarray,
    level_idx: np.ndarray,
    season_idx: np.ndarray,
    pitcher_idx: np.ndarray,
    *,
    feature_list: list[str] | None = None,
    mu_mean: float  = 88.0,
    mu_sd:   float  = 30.0,
    sigma_prior: float = 10.0,
    draws: int      = 200,
    tune:  int      = 200,
    target_accept: float = 0.9,
    verbose: bool   = True,
    sampler: str    = "jax",
    chains: int     = 1,
    monitor_memory: bool = True,
    force_memory_allocation: bool = True,
    allocation_target: float = 0.8,
    direct_feature_input: tuple = None,  # (X, y, feature_names) for testing
):
    cols = _ColumnSchema()
    if feature_list is None:
        feature_list = cols.model_features()

    # Take initial memory snapshot if monitoring enabled
    if monitor_memory:
        take_memory_snapshot("Before model setup")
        
    # Force memory allocation if requested
    if force_memory_allocation and monitor_memory:
        if verbose:
            print("\n=== Pre-training Memory Allocation ===")
        allocation_result = force_allocation_if_needed(
            target_fraction=allocation_target,
            current_usage_threshold=0.4,
            step_size_mb=1000,
            max_steps=8,
            verbose=verbose
        )
        if verbose:
            print(f"Memory allocation result: {json.dumps(allocation_result, indent=2)}")

    # 1) design matrix - either from transformer or direct input
    if direct_feature_input is not None:
        # Use direct input (for testing)
        X, y, feature_names = direct_feature_input
        if feature_list is not None:
            # Filter features if list provided
            selected_indices = [i for i, name in enumerate(feature_names) if name in feature_list]
            X = X[:, selected_indices]
            feature_names = [feature_names[i] for i in selected_indices]
    else:
        # Use transformer
        X_all, y_ser = transform_preprocessor(df_raw, transformer)
        names = transformer.get_feature_names_out()
        X = X_all[:, np.isin(names, feature_list)]
        y = y_ser.values
        feature_names = [fn for fn in names if fn in feature_list]

    # Cast indices to Python ints
    n_bat  = int(batter_idx.max()) + 1
    n_lvl  = int(level_idx.max()) + 1
    n_seas = int(season_idx.max()) + 1
    n_pit  = int(pitcher_idx.max()) + 1
    n_feat = int(X.shape[1])

    if verbose:
        print(f"🔍 Data dims: n_bat={n_bat}, n_lvl={n_lvl}, n_seas={n_seas}, n_pit={n_pit}, n_feat={n_feat}")

    with pm.Model() as model:
        # Priors
        mu         = pm.Normal("mu", mu_mean, mu_sd)
        beta_level = pm.Normal("beta_level", 0, 5, shape=n_lvl)
        beta       = pm.Normal("beta",       0, 5, shape=n_feat)

        sigma_b    = pm.HalfNormal("sigma_b", sigma_prior)
        u_raw      = pm.Normal("u_raw", 0, 1, shape=n_bat)
        u          = pm.Deterministic("u", u_raw * sigma_b)

        sigma_seas = pm.HalfNormal("sigma_seas", sigma_prior)
        v_raw      = pm.Normal("v_raw", 0, 1, shape=n_seas)
        v          = pm.Deterministic("v", v_raw * sigma_seas)

        sigma_pit  = pm.HalfNormal("sigma_pit", sigma_prior)
        p_raw      = pm.Normal("p_raw", 0, 1, shape=n_pit)
        p          = pm.Deterministic("p", p_raw * sigma_pit)

        # Likelihood
        theta = (
            mu
            + beta_level[level_idx]
            + pm.math.dot(X, beta)
            + u[batter_idx]
            + v[season_idx]
            + p[pitcher_idx]
        )
        sigma_e  = pm.HalfNormal("sigma_e", sigma_prior)
        pm.Normal("y_obs", theta, sigma_e, observed=y)

        # Memory snapshot before sampling if monitoring enabled
        if monitor_memory:
            take_memory_snapshot("Before sampling")
            # Take another memory snapshot after a small delay to catch any lazy allocation
            time.sleep(1)
            take_memory_snapshot("Before sampling (after delay)")

        # Sampling with timing and memory monitoring
        sampling_context = (
            monitor_memory_usage("MCMC Sampling", verbose=verbose) if monitor_memory 
            else _timed_section("compile+sample")
        )
        
        with sampling_context:
            idata = pm.sample(
                draws=draws,
                tune=tune,
                chains=chains,
                target_accept=target_accept,
                nuts_sampler="numpyro" if (sampler=="jax" and USE_JAX) else "nuts",
                **(
                    {"nuts_sampler_kwargs": {"chain_method": "vectorized"}}
                    if (sampler == "jax" and USE_JAX)
                    else {}
                ),
                progressbar=verbose,
                idata_kwargs={"log_likelihood": ["y_obs"]},
            )

        # Memory snapshot after sampling if monitoring enabled
        if monitor_memory:
            take_memory_snapshot("After sampling")

        # Posterior predictive timing with memory monitoring
        posterior_context = (
            monitor_memory_usage("Posterior Predictive", verbose=verbose) if monitor_memory 
            else _timed_section("posterior_predictive")
        )
        
        with posterior_context:
            idata.extend(pm.sample_posterior_predictive(idata, var_names=["y_obs"]))

    # Store feature names
    idata.attrs["feature_names"] = feature_names

    if verbose:
        print("⚡ Posterior predictive samples (first 5):",
              idata.posterior_predictive["y_obs"]
                   .stack(s=("chain", "draw")).values[:5])

    # Generate memory report if monitoring enabled
    if monitor_memory:
        report = generate_memory_report("hierarchical_memory_report.json")
        if "summary" in report and "gpu_utilization" in report["summary"]:
            util = report["summary"]["gpu_utilization"]
            print("\n=== Memory Usage Summary ===")
            print(f"GPU Utilization - Min: {util['min']:.2f}% Max: {util['max']:.2f}% Avg: {util['avg']:.2f}%")
            print(f"Detailed report saved to hierarchical_memory_report.json")

    return idata


# ------------------------------------------------------------------





# ───────────────────────────────────────────────────────────────────────
# 6. Smoke test (only run when module executed directly)
# ───────────────────────────────────────────────────────────────────────
if __name__ == "__main__":

    from pathlib import Path
    import pandas as pd, numpy as np, arviz as az
    from src.data.load_data import load_and_clean_data
    from src.features.feature_engineering import feature_engineer
    from src.features.preprocess import (fit_preprocessor,
                                        prepare_for_mixed_and_hierarchical)
    # from src.models.hierarchical import fit_bayesian_hierarchical
    from src.utils.hierarchical_utils import save_model, save_preprocessor
    from src.utils.posterior import posterior_to_frame
    from src.utils.bayesian_metrics import (compute_classical_metrics,
                                            compute_bayesian_metrics,
                                            compute_convergence_diagnostics,
                                            compute_calibration)

    from src.utils.validation import (
        run_kfold_cv_pymc,
        rmse_pymc,
        posterior_predictive_check,
    )
    import json  # Added import for JSON operations
    
    RAW   = Path("data/Research Data Project/Research Data Project/exit_velo_project_data.csv")
    OUT_NC = Path("data/models/saved_models/exitvelo_hmc.nc")
    OUT_POST = Path("data/models/saved_models/posterior_summary.parquet")
    OUT_PREPROC = Path("data/models/saved_models/preprocessor.joblib")

    # 1 · prep
    df = load_and_clean_data(RAW)
    df_fe = feature_engineer(df)
    df_model = prepare_for_mixed_and_hierarchical(df_fe)

    _, _, tf = fit_preprocessor(df_model, model_type="linear", debug=False)

    b_idx = df_model["batter_id"].cat.codes.values
    l_idx = df_model["level_idx"].values
    s_idx = df_model["season_idx"].values
    p_idx = df_model["pitcher_idx"].values
    draws_and_tune = 50
    target_accept=0.95
    chains=4
    # 2 · fit
    idata = fit_bayesian_hierarchical(
        df_model,
        tf,
        b_idx,            # batter codes
        l_idx,            # level codes
        s_idx,            # season codes
        p_idx,            # pitcher codes
        sampler="jax",
        draws=draws_and_tune,
        tune=draws_and_tune,
        target_accept=target_accept,
        chains=chains,
        monitor_memory=True,  # Enable memory monitoring
        force_memory_allocation=True,  # Force memory allocation
        allocation_target=0.8,  # Target 80% memory utilization
        direct_feature_input=None  # No direct feature input for this example
    )



    idata.attrs["median_age"] = df_model["age"].median()   # ← NEW

    # 3 · persist
    save_model(idata, OUT_NC)
    save_preprocessor(tf, OUT_PREPROC)
    posterior_to_frame(idata).to_parquet(OUT_POST)
    print("✅ training complete – artefacts written:")
    print("   •", OUT_NC)
    print("   •", OUT_POST)
    print("   •", OUT_PREPROC)


    # # ─── NEW: IMPORT VALIDATION UTILITIES ─────────────────────────────
    # # ─── 7) Sanity‐check CV on your real data ───────────────────────────
    # print("\n--- PyMC hierarchical CV on real data ---")
    # cv_scores = run_kfold_cv_pymc(
    #     lambda d: fit_bayesian_hierarchical(
    #         d, tf, b_idx, l_idx, s_idx, p_idx,
    #         sampler="jax",
    #         draws=draws_and_tune,
    #         tune=draws_and_tune,
    #         target_accept=target_accept,
    #         verbose=False,  # silent inside CV
    #         chains = chains
    #     ),
    #     df_model,
    #     k=3
    # )
    # print("CV RMSEs:", cv_scores)


    # Extract the true target vector for classical metrics
    _, y_full = transform_preprocessor(df_model, tf)
    print("=== Classical Metrics ===")
    compute_classical_metrics(idata, y_full.values)

    # print("\n=== Calibration ===")
    # compute_calibration(idata, y_full.values)

    print("\n=== Bayesian Metrics ===")
    compute_bayesian_metrics(idata)

    print("\n=== Convergence Diagnostics ===")
    compute_convergence_diagnostics(idata)


    # # ─── 8) RMSE & PPC on the full training set ─────────────────────────
    # print("\n--- rmse_pymc on full training data ---")
    # rmse_full = rmse_pymc(idata, df_model)
    # print("RMSE (train):", rmse_full)

    # print("\n--- posterior_predictive_check (real data) ---")
    # fig = posterior_predictive_check(idata, df_model, batter_idx=b_idx)
    # fig.savefig("ppc_real_data.png")
    # print("Saved plot to ppc_real_data.png")



Overwriting src/models/hierarchical.py


In [26]:
%%writefile src/models/hierarchical_predict.py


import json
from pathlib import Path

import arviz as az
import pandas as pd
import numpy as np

from src.utils.posterior import align_batter_codes

# ─── new helper at top of file ────────────────────────────────────────────
def get_top_hitters(
    df: pd.DataFrame,
    hitter_col: str = "hitter_type",
    n: int = 5,
    verbose: bool = False
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Given a DataFrame with predictions and a 'hitter_type' column,
    return two DataFrames of the top-n POWER and CONTACT hitters by pred_mean.
    If the column is missing, returns two empty DataFrames (and prints a warning).
    """
    if hitter_col not in df.columns:
        if verbose:
            print(f"[Warning] Column '{hitter_col}' not found; skipping top-hitter extraction.")
        empty = pd.DataFrame(columns=[ "batter_id", hitter_col, "pred_mean", "pred_lo95", "pred_hi95" ])
        return empty, empty

    # 1) Filter power vs. contact, case‐insensitive
    power_mask   = df[hitter_col].str.contains("POWER",   case=False, na=False)
    contact_mask = df[hitter_col].str.contains("CONTACT", case=False, na=False)

    power_df   = df.loc[power_mask]
    contact_df = df.loc[contact_mask]

    # 2) Take the top-n by pred_mean
    top_power   = power_df.nlargest(n, "pred_mean")[ ["batter_id", hitter_col, "pred_mean", "pred_lo95", "pred_hi95"] ]
    top_contact = contact_df.nlargest(n, "pred_mean")[ ["batter_id", hitter_col, "pred_mean", "pred_lo95", "pred_hi95"] ]

    if verbose:
        print(f"[Top {n}] power hitters:\n{top_power}")
        print(f"[Top {n}] contact hitters:\n{top_contact}")

    return top_power, top_contact



def simplified_prepare_validation(df_val: pd.DataFrame, median_age: float, verbose: bool = False) -> pd.DataFrame:
    """
    Prepare validation dataset with simplified approach, handling missing columns gracefully.

    Assumptions:
    - All predictions are for MLB-level competition (level_idx = 2).
    - Missing age defaults to median training age (age_centered = 0).

    Parameters
    ----------
    df_val : pd.DataFrame
        Raw validation dataframe (must include 'season', 'batter_id', optional 'age').
    median_age : float
        Median age computed from training data for centering.
    verbose : bool
        If True, prints debug information.

    Returns
    -------
    pd.DataFrame
        DataFrame with added 'age_centered' and 'level_idx'.
    """
    df_val = df_val.copy()

    # Default to MLB level for all observations
    df_val['level_idx'] = 2

    # Center age
    if 'age' in df_val.columns:
        df_val['age_centered'] = df_val['age'] - median_age
    else:
        # Using median training age => zero effect
        df_val['age_centered'] = 0.0

    if verbose:
        print(f"[Validation Prep] level_idx set to 2 for all rows, age_centered stats: min={df_val['age_centered'].min()}, max={df_val['age_centered'].max()}")

    return df_val


def predict_from_summaries(
    roster_csv: Path,
    posterior_parquet: Path,
    global_effects_json: Path,
    output_csv: Path,
    verbose: bool = False
) -> pd.DataFrame:
    """
    Load saved model summaries and raw roster, merge random effects,
    compute point predictions and intervals for validation set,
    write out predictions CSV, and return the DataFrame.

    Returns
    -------
    pd.DataFrame
        DataFrame with columns:
        ['season','batter_id','age','level_idx','age_centered','batter_idx',
         'u_q50','u_q2.5','u_q97.5',
         'contrib_age','contrib_level','contrib_u','contrib_features',
         'pred_mean','pred_lo95','pred_hi95']
    """
    # 1) Load posterior random effects (batter-level)
    df_post = pd.read_parquet(posterior_parquet)
    if verbose:
        print(f"[Load] posterior_summary: {df_post.shape} rows")

    # 2) Load validation roster
    df_roster = pd.read_csv(roster_csv)
    if verbose:
        print(f"[Load] validation roster: {df_roster.shape} rows, columns: {df_roster.columns.tolist()}")

    # 3) Load global effects
    glob        = json.loads(global_effects_json.read_text())
    post_mu     = glob['mu_mean']
    beta_age    = glob['beta_age']
    beta_level  = glob['beta_level'][2]  # MLB-level coefficient
    med_age     = glob['median_age']
    if verbose:
        print(f"[Global Effects] mu={post_mu}, beta_age={beta_age}, "
              f"beta_level={beta_level}, median_age={med_age}")

    # 4) Prepare minimal validation features
    df_val = simplified_prepare_validation(df_roster, med_age, verbose)

    # 5) Align batter codes and merge random effects
    df_val['batter_idx'] = align_batter_codes(df_val, df_post['batter_idx'])
    df = df_val.merge(df_post, on='batter_idx', how='left')
    if verbose:
        print(f"[Merge] merged validation with posterior: {df.shape}")

    # 6) Compute contributions & predictions
    # 6a) Age contribution
    df['contrib_age']   = beta_age * df['age_centered']
    # 6b) Level (MLB) contribution
    df['contrib_level'] = beta_level
    # 6c) Batter random effect (median)
    df['contrib_u']     = df['u_q50']

    # 6d) Point‐estimate
    df['pred_mean']  = post_mu + df['contrib_age'] + df['contrib_level'] + df['contrib_u']

    # 6e) 95% credible‐interval bounds
    df['pred_lo95']  = post_mu + df['contrib_age'] + df['contrib_level'] + df['u_q2.5']
    df['pred_hi95']  = post_mu + df['contrib_age'] + df['contrib_level'] + df['u_q97.5']

    # 6f) Placeholder for any other feature contributions
    df['contrib_features'] = 0.0

    # 7) Persist predictions CSV
    df.to_csv(output_csv, index=False)
    if verbose:
        print(f"[Save] Predictions written to {output_csv}")

    # 8) Return for downstream inspection or metrics
    return df




if __name__ == '__main__':
    from src.utils.validation import (
        prediction_interval,
        bootstrap_prediction_interval,
    )

    BASE    = Path('data/models/saved_models')
    SUMMARY = BASE / 'posterior_summary.parquet'
    GLOBAL  = BASE / 'global_effects.json'
    ROSTER  = Path('data/Research Data Project/Research Data Project/exit_velo_validate_data.csv')
    OUT     = Path('data/predictions/exitvelo_predictions_2024.csv')

    # 1) Generate predictions + CI columns
    predict_df = predict_from_summaries(
        roster_csv=ROSTER,
        posterior_parquet=SUMMARY,
        global_effects_json=GLOBAL,
        output_csv=OUT,
        verbose=True
    )
    print(predict_df.head())

    # 2) Empirical RMSE if 'exit_velo' present
    df_val = pd.read_csv(ROSTER)
    if 'exit_velo' in df_val.columns:
        y_true = df_val['exit_velo'].values
        y_pred = predict_df['pred_mean'].values
        rmse_val = np.sqrt(np.mean((y_pred - y_true)**2))
        print("\n--- empirical RMSE on validation set ---", rmse_val)

    # 3) Prepare a preds DataFrame for CI routines
    preds = predict_df['pred_mean'].to_frame(name='pred')
    lo95  = predict_df['pred_lo95'].values
    hi95  = predict_df['pred_hi95'].values


    # 4) Safely extract top hitters (requires a 'hitter_type' column)
    top_power, top_contact = get_top_hitters(predict_df, hitter_col="hitter_type", n=5, verbose=True)

    print("\nTop Power Hitters (if any):\n", top_power)
    print("\nTop Contact Hitters (if any):\n", top_contact)
    

Overwriting src/models/hierarchical_predict.py


In [27]:
%%writefile src/train.py
"""
Train / compare four families on a 70‑30 split.

Run:
    python -m src.train
"""
from __future__ import annotations
import pandas as pd
from sklearn.model_selection import train_test_split

from src.data.load_data import load_and_clean_data
from src.features.preprocess import preprocess

from src.models.linear import fit_ridge
from src.models.gbm   import fit_gbm
from src.models.mixed import fit_mixed
from src.models.hierarchical import fit_bayesian_hierarchical

RAW_PATH = "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"


def main():
    df_raw   = load_and_clean_data(RAW_PATH)
    df_clean = preprocess(df_raw)

    train_df, test_df = train_test_split(
        df_clean, test_size=0.30, random_state=42, stratify=df_clean["level_abbr"]
    )

    # ––– A  Ridge  –––––––––––––––––––––––––––––––
    _, rmse_ridge = fit_ridge(train_df, test_df)
    print(f"Ridge RMSE ……  {rmse_ridge:5.2f} mph")

    # ––– B  Gradient‑Boost  ––––––––––––––––––––––
    _, rmse_gbm = fit_gbm(train_df, test_df)
    print(f"XGBoost RMSE … {rmse_gbm:5.2f} mph")

    # ––– C  Mixed‑Effects  –––––––––––––––––––––––
    _, rmse_mixed = fit_mixed(train_df, test_df)
    print(f"Mixed‑LM RMSE  {rmse_mixed:5.2f} mph")

    # ––– D  Bayesian Hierarchical (quick sample) –
    idata = fit_bayesian_hierarchical(
        train_df,
        batter_idx=train_df.batter_id.astype("category").cat.codes.values,
        level_idx=train_df.level_idx.values,
        age_centered=train_df.age_centered.values,
        mu_prior=90,
        sigma_prior=5,
        draws=500, tune=500   # short run for demo
    )
    from src.utils.validation import rmse_pymc
    rmse_bayes = rmse_pymc(idata, test_df)
    print(f"PyMC RMSE ……  {rmse_bayes:5.2f} mph")


if __name__ == "__main__":
    main()



Overwriting src/train.py


In [28]:
%%writefile src/utils/dash_compat.py
# src/utils/dash_compat.py
import dash
import functools
import dash_bootstrap_components as dbc

# ── 3.0 compat: if Dash.run_server() was removed, alias it to run()
if not hasattr(dash.Dash, "run_server"):
    dash.Dash.run_server = dash.Dash.run

_DROPDOWN_PATCH_KEY = "_ghadf_dropdown_patched"

def patch_dropdown_right_once() -> None:
    """
    Back-compat shim: translate deprecated `right=` → `align_end=`,
    and patch it exactly once per Python process.
    """
    if getattr(dbc, _DROPDOWN_PATCH_KEY, False):
        return

    orig_init = dbc.DropdownMenu.__init__

    @functools.wraps(orig_init)
    def _init(self, *args, **kwargs):
        if "right" in kwargs and "align_end" not in kwargs:
            kwargs["align_end"] = kwargs.pop("right")
        return orig_init(self, *args, **kwargs)

    dbc.DropdownMenu.__init__ = _init
    setattr(dbc, _DROPDOWN_PATCH_KEY, True)



Overwriting src/utils/dash_compat.py


In [29]:
%%writefile src/utils/dash_utils.py

import dash
from src.utils.dash_compat import patch_dropdown_right_once
import pandas as pd
import dash_bootstrap_components as dbc
from explainerdashboard import RegressionExplainer, ExplainerDashboard
from __future__ import annotations
import socket, contextlib, time

def _in_notebook() -> bool:
    try:
        from IPython import get_ipython
        ip = get_ipython()
        return ip is not None and ip.has_trait("kernel")
    except Exception:
        return False

# Apply necessary shims at import time
patch_dropdown_right_once()



def _port_in_use(host: str, port: int) -> bool:
    with contextlib.closing(socket.socket()) as s:
        s.settimeout(0.001)
        return s.connect_ex((host, port)) == 0

def _get_free_port() -> int:
    with contextlib.closing(socket.socket()) as s:
        s.bind(("", 0))
        return s.getsockname()[1]
    
def launch_explainer_dashboard(
    pipeline,
    preprocessor,
    X_raw: pd.DataFrame,
    y: pd.Series,
    *,
    cats=None,
    descriptions=None,
    title: str = "Model Explainer",
    bootstrap=dbc.themes.FLATLY,
    inline: bool | None = None,
    host: str = "127.0.0.1",
    port: int = 8050,
    **db_kwargs,
):
    """Create & display an ExplainerDashboard.

    • Inline in notebooks by default.  
    • Auto-terminates any previous inline dashboard on the same port.  
    • Falls back to a free port if the requested one is still busy.
    """
    if inline is None:
        inline = _in_notebook()

    # ---------- tidy up any existing inline server ----------
    if inline and _port_in_use(host, port):
        try:
            ExplainerDashboard.terminate(port)
        except Exception:             # pragma: no cover
            pass                      # nothing running or unsupported version
        # give the OS a moment to release the socket
        for _ in range(10):
            if not _port_in_use(host, port):
                break
            time.sleep(0.1)
        else:                          # still busy after 1 s
            port = _get_free_port()

    # ---------- prepare dataset ----------
    X_proc     = preprocessor.transform(X_raw)
    feat_names = preprocessor.get_feature_names_out()
    X_df       = pd.DataFrame(X_proc, columns=feat_names, index=X_raw.index)

    if cats:
        cats = [c for c in cats if any(fn.startswith(f"{c}_") for fn in feat_names)]

    explainer = RegressionExplainer(
        pipeline,
        X_df,
        y,
        cats=cats or [],
        descriptions=descriptions or {},
        precision="float32",
    )
    explainer.merged_cols = explainer.merged_cols.intersection(X_df.columns)

    # ---------- launch dashboard ----------
    if inline:
        ExplainerDashboard(
            explainer,
            title=title,
            bootstrap=bootstrap,
            mode="inline",
            **db_kwargs,
        ).run(host=host, port=port)
    else:
        ExplainerDashboard(
            explainer,
            title=title,
            bootstrap=bootstrap,
            **db_kwargs,
        ).run(host=host, port=port)







if __name__=="__main__":
    from pathlib import Path
    import pandas as pd
    from src.data.load_data import load_and_clean_data
    from src.features.feature_engineering import feature_engineer
    from src.features.preprocess import transform_preprocessor
    from src.utils.gbm_utils import load_pipeline
    from src.data.ColumnSchema import _ColumnSchema

    # 1) load your trained pipeline + preprocessor
    model_pipeline, preprocessor = load_pipeline("data/models/saved_models/gbm_pipeline.joblib")

    # 2) load & prepare a small sample
    df_raw = load_and_clean_data(
        "data/Research Data Project/Research Data Project/exit_velo_project_data.csv"
    ).sample(200, random_state=42)
    df_fe  = feature_engineer(df_raw)
    X_raw = df_fe.drop(columns=["exit_velo"])
    y_raw = df_fe["exit_velo"]

    # 3) category grouping helper
    cols = _ColumnSchema()

    # 4) launch the dashboard on port 8050
    launch_explainer_dashboard(
        pipeline      = model_pipeline,
        preprocessor  = preprocessor,
        X_raw         = X_raw,
        y             = y_raw,
        cats          = cols.nominal(),
        descriptions  = {c: c for c in preprocessor.get_feature_names_out()},
        whatif        = True,
        shap_interaction = False,
        hide_wizard     = True
    )


Overwriting src/utils/dash_utils.py
